<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/791112_50000_shuffled_ep8_alpha10_gamma15_micro_samples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time

# Wait until /content is writable — sign that Colab is ready
print("⏳ Waiting for runtime to be ready...", end="", flush=True)

while not os.path.exists("/content") or not os.access("/content", os.W_OK):
    time.sleep(0.5)
    print(".", end="", flush=True)

print("\n✅ Runtime is ready. Continuing...")


⏳ Waiting for runtime to be ready...
✅ Runtime is ready. Continuing...


### Datasets and Transformers
- 2 Hugging Face libraries
- well integrated with each other
- Datasets also well integrated with PyTorch and TensorFlow libraries

In [ ]:
!pip install -U transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 131.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0


### Update the Hugging Face library 'datasets'; ignore the following message
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia...
gcsfs 2025.3.2 requires fsspec

- Upgrade: datasets 2.14.4 => datasets-3.6.0 because 2.14.4 has bugs with Path
- I don't use gcsfs


In [ ]:
!pip install -U datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
!pip install -q accelerate
!pip install -q huggingface_hub
!pip install -q huggingface_hub[hf_xet]
!pip install -q scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.0 MB/s eta 0:00:00


In [ ]:
import json
import logging
import numpy as np
import os
import pandas as pd
import pickle
import shutil
import sys
import time
import torch
import torch.nn.functional as F
import wandb
import zipfile

from contextlib                    import suppress
from datasets                      import Dataset, DatasetDict, load_from_disk, load_dataset
from datetime                      import datetime, timezone
from google.colab                  import auth, drive, files, runtime, userdata
from huggingface_hub               import create_branch, create_repo, HfApi, login, upload_file, hf_hub_download, whoami
from huggingface_hub.errors        import RepositoryNotFoundError
from pathlib                       import Path
from sklearn.metrics               import f1_score, precision_score, recall_score, roc_auc_score, average_precision_score, accuracy_score, hamming_loss, classification_report,  precision_recall_fscore_support
from torch.optim                   import AdamW
from torch.utils.data              import DataLoader
from tqdm.auto                     import tqdm
from transformers                  import LongformerTokenizerFast, LongformerForSequenceClassification
from transformers.trainer_utils    import EvalPrediction
from transformers.trainer          import Trainer
from transformers.training_args    import TrainingArguments
from transformers.trainer_callback import TrainerCallback
#from transformers                  import logging as transformers_logging
from torch.nn                      import BCEWithLogitsLoss, Module
from zoneinfo                      import ZoneInfo


### Configuration
1. UPDATE THE ipynb FILENAME
2. GIVE ACCESS TO HF
3. UPDATE THE HF HUB REPO_ID
4. UPDATE THE MODEL CARD



---


#### epoch
- 1 epoch is a complete pass through the entire training dataset
- with n datapoints and batch size = b, n/b iterations to complete 1 epoch
- 1 iteration is a single update of the model's parameters (weights, etc.)


---


#### repo_id, branch, model and dataset repos on HF Hub
- 1 repo = 1 model and 1 tokenizer
- branch = revision



---

#### Averages
<pre>
- per sample: metrics are computed for each sample (= for each instance, = for each row of y_true and y_pred), and then averaged across all samples.
- per label : metrics are computed for each label separately, and then averaged across all labels.
- per batch : metrics are computed for each batch, and then averaged across all batches.

- 'micro': Calculate metrics globally by counting the total true positives, false negatives and false positives.
           Gives more weight to frequent labels → best for imbalanced datasets where frequent labels are more important.
- 'macro': Calculate metrics for each label separately, and find their unweighted mean.
           This does not take label imbalance into account. This is fine for balanced datasets but not for imbalanced datasets since rare labels are given equal weights.
           Averages the metric for each label without considering their imbalance, without considering label frequency.
           When to use: when wanting equal importance for all labels, including rare ones.
           Treats all labels equally → best when you care about rare labels as much as frequent ones.
- 'weighted': Calculate metrics for each label separately, and find their average weighted by support (= the number of true instances for each label). This alters ‘macro’ to account for label imbalance;
              it can result in an F-score that is not between precision and recall.
              Averages per label weighted by their support, without considering label frequency.
              When to use: when wanting to reflect label imbalance (common labels contribute more).
              Like macro but considers label frequency → best if you want a compromise between macro and micro.
- 'samples': Calculate metrics per sample instead of per label, and find their average (only meaningful for multilabel-classification where this differs from accuracy_score).
             Computes the metric per sample and then averages across all samples.
             When to use: when each sample has multiple correct labels.

- 'macro' or 'weighted' AUC is often best because AUC isn't as affected by class imbalance as F1/Precision/Recall
- 'macro'      AUC: usually the best because it treats all labels equally, avoiding the dominance of frequent labels
- 'weighted'   AUC: similar to macro but considers label frequency
- 'macro'   PR AUC: best for imbalanced datasets because it treats rare labels fairly
- 'weighted PR AUC: also good, but slightly biased toward frequent labels

PR AUC is better than ROC AUC when you care about positive examples in imbalanced data.
</pre>


---

#### learning rate
- A common rule is to scale the learning rate proportionaly with the effective batch size
- **>>>>>>>>>> get_linear_schedule_with_warmup**

#### Path separators
Use os.path.join for constructing paths to ensure correct separators

### ml-poc => load_datasetdict_from_zip_to_import.py

In [ ]:
uploaded = files.upload()


Saving load_datasetdict_from_zip_to_import.py to load_datasetdict_from_zip_to_import.py


In [ ]:
sys.path.append('/content')
import load_datasetdict_from_zip_to_import


### Define literal data

In [ ]:
THRESHOLD_TUNING = True
ACCESS_TO_HF_HUB = True


In [ ]:
skills         = 791112
all_rows_low   = 0
all_rows_high  = 50000 # 120 1200 12000 24000 36000 48000
num_datapoints = all_rows_high - all_rows_low

epochs = 1  #8 #16 #14 #12 #10

local_tz  = ZoneInfo("Europe/Brussels")
#timestamp = f"{datetime.now(local_tz).strftime('%Y-%m-%d_%H-%M')}"
#timestamp = f"{datetime.now(local_tz).strftime('%Y-%m-%d_%H')}"
timestamp = f"{datetime.now(local_tz).strftime('%Y-%m-%d')}"

# repo_id (str) = namespace(user or organization)/repo name
repo_id   = f"claudelepere/jobs_EN_791112_50000_all_{timestamp}"
print(f"repo_id: {repo_id}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

training_average   = 'micro'
evaluation_average = 'samples'

learning_rate = 2e-5 #1e-5 x 32/8

# default threshold = 0.5
threshold = 0.5

# attention window size
attention_window = 1024 #512

#alfa  = 0.5
#gamma = 4.0
#alpha = 0.75
#gamma = 2.0
alpha = 1.0
gamma = 1.5

multi_label_metrics_counter = 0
compute_metrics_counter     = 0
compute_loss_counter        = 0


repo_id: claudelepere/jobs_EN_791112_50000_all_2025-07-04
device: cuda


### OOM = Out Of Memory

#### OOM: use fp16 (half precision) mixed precision training
Reduces memory requirements by up to 50%.

fp16 = True in TrainingArguments

#### OOM: reduce batch size
- small sizes (1 to 32):
  - PROs: better generalization in some cases
  - CONs: may produce noisier gradients
- large sizes (128, 256, or higer):
  - PROs: gradients are smoother, leading to more stable training
  - CONs: poorer generalization (overfitting) in some cases
- intermediate sizes (32, 64): combines the benefits of small and large sizes

In [ ]:
batch_size = 8

#### OOM: enable gradient accumulation

- compensate for smaller batch sizes by accumulating gradients over several steps
- **effective batch size** = per-device batch size x gradient acumulation steps
- in each iteration, the model computes the gradients, these gradients are immediately used to update the model parameters

WARNING: gradient_accumulation_steps may not be None => comment it in TrainingArguments

In [ ]:
gradient_accumulation_steps = 4

#### OOM: use PYTORCH_CUDA_ALLOC_CONF to handle memory fragmentation

In [ ]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

#### OOM: reduce model size or input tokens
- LongformerTokenizer.from_pretrained('allenai/longformer-base/large-4096'): large/base: 435M/149M parameters
- max_length: 4096 max for Longformer
- a single word can be equal to several tokens; stop words are **NOT discarded**!
- word_text_length_counts_sorted:
      jobs count                 : 50000
      jobs count under  512 words: 44794  89.59%
      jobs count under  640 words: 47894  95.79%
      jobs count under  768 words: 49123  98.25%
      jobs count under  896 words: 49691  99.38%
      jobs count under 1024 words: 49917  99.83%
      jobs count under 2048 words: 50000 100.00%
      jobs count under 4096 words: 50000 100.00%

In [ ]:
#max_length =  768    #      37 min    #
max_length = 1024    #      38 min    # GPU RAM: 12.2 / 40 GB <<<<<<<<<<<< pourrait augmenter à 2048?
#max_length = 2048    # 1 hr 10 min    # GPU RAM: 21.4 / 40 GB
#max_length = 4096    # 2 hr 10 min    # GPU RAM: 39.5 / 40 GB => OutOfMemoryError

#### OOM: limit the number of GPU workers:
* 0 (default) or 1
* in Colab dataloader_num_workers = 1

#### """OOM: reduce the number of transformers layers"""

In [ ]:
"""
hidden_layers = 6  # default:12
"""

'\nhidden_layers = 6  # default:12\n'

#### OOM: check for and kill zombie processes

In [ ]:
!ps aux | grep python
#!kill -9 <PID>
if torch.cuda.is_available():
  !nvidia-smi
  print(torch.cuda.memory_summary())

root          85  0.7  0.0      0     0 ?        Z    16:29   0:12 [python3] <defunct>
root          86  0.0  0.0  77372 57328 ?        S    16:29   0:00 python3 /usr/local/bin/colab-fileshim.py
root         131  0.2  0.1 798720 135232 ?       Sl   16:29   0:04 /usr/bin/python3 /usr/local/bin/jupyter-notebook --debug --transport="ipc" --ip=172.28.0.12 --NotebookApp.token= --port=9000 --FileContentsManager.root_dir=/ --MappingKernelManager.root_dir=/content
root        6029 17.3  1.5 12556128 1382916 ?    Ssl  16:53   0:35 /usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-313f22bb-0bc6-42b8-9c94-76f295ecf9cb.json
root        6065  1.0  0.0 546836 22152 ?        Sl   16:53   0:02 /usr/bin/python3 /usr/local/lib/python3.11/dist-packages/debugpy/adapter --for-server 46175 --host 127.0.0.1 --port 38157 --server-access-token c5c086bf8aa8f05c2fd9cc759dc2de94828e3ffd1dd5ff28d5cf20ec51519390
root        7024  0.0  0.0 1243872 21192 ?       Sl   16:55   0:00 

#### OOM: free up GPU memory

In [ ]:
torch.cuda.empty_cache()

### Hugging Face model card

In [ ]:
# HF model card
# Model card here => README.md on the HF Hub.

model_card = """
---
tags:
- '50000'
---
# Model
Model fine-tuned on higly imbalanced multilabel classification.

## Model details
- Language: English
- Task: Multilabel classification
- Architecture: Longformer
- Pretrained model: [allenai/longformer-base-4096](https://huggingface.co/allenai/longformer-base-4096)
- Framework: Pytorch
- Version 1.0.0

## Training Data
- skills: 7, 9, 11, 12
- 50000 job datapoints

## Fine-tuning parameters
- batch size: 8
- gradient accumulation: 4
- fp16 precision
- input tokens max length: 1024
- epochs: 8
- learning rate: 2e-5
- attention window size: 1024
- training average: micro
- evaluation average: samples
- threshold tuning: True
- threshold: 0.5
- focal loss: alpha=1.0, gamma=1.5
"""

### Input
**NO_PUSH** =>
1. dataset_EN_791112_0_50000_all.zip


In [ ]:
datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}_all.zip"
#if above_threshold:
#    datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}_above.zip"
#else:
#    datasetDict_zip_file_name = f"dataset_EN_{skills}_{all_rows_low}_{all_rows_high}_below.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]

print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")


datasetDict_zip_file_name: dataset_EN_791112_0_50000_all.zip
datasetDict_dir_name     : dataset_EN_791112_0_50000_all


### Output

=> **HF Hub**
- Model Repo Url: https://huggingface.co/claudelepere/jobs_EN_791112_50000_all_mmddhh
- Dataset Repo Url: https://huggingface.co/datasets/claudelepere/jobs_EN_791112_50000_all_mmddhh
- Model card => https://huggingface.co/claudelepere/jobs_EN_791112_50000_all_mmddhh
___
=> **HF Hub model repo**
1. labels = 'labels', 'id2label'
2. tokenizer
3. model
4. optimized_thresholds
5. trainer_train
6. evaluation_trainer_evaluate
7. evaluation_trainer_predict
8. evaluation_model_eval
9. test_trainer_evaluate
10. test_trainer_predict
11. test_model_eval

=> **HF Hub dataset repo**
1. encoded_validation_dataset

In [ ]:
encoded_validation_dataset_tmp        = "encoded_validation_dataset_tmp"
encoded_validation_dataset_zip        = "encoded_validation_dataset.zip"
encoded_validation_dataset            = os.path.splitext(encoded_validation_dataset_zip)[0]
#encoded_validation_dataset_drive_path = "/content/drive/MyDrive/encoded_validation_dataset"
#encoded_validation_dataset_file_path  = f"{encoded_validation_dataset_drive_path}/{encoded_validation_dataset_zip}"


## Ensure the directory exists before saving the dataset and making the archive
#os.makedirs(encoded_validation_dataset_tmp, exist_ok=True)


print(f"encoded_validation_dataset_tmp : {encoded_validation_dataset_tmp}")
print(f"encoded_validation_dataset_zip : {encoded_validation_dataset_zip}")
print(f"encoded_validation_dataset     : {encoded_validation_dataset}")
#print(f"encoded_validation_dataset_drive_path      : {encoded_validation_dataset_drive_path}")
#print(f"encoded_validation_dataset_file_path       : {encoded_validation_dataset_file_path}")


encoded_validation_dataset_tmp : encoded_validation_dataset_tmp
encoded_validation_dataset_zip : encoded_validation_dataset.zip
encoded_validation_dataset     : encoded_validation_dataset


### """Delete a file if present"""

In [ ]:
"""
def delete_local_file(file_path):
    try:
        file_path.unlink(missing_ok=True)  # missing_ok=True prevents exception if file doesn't exist
        print(f"Successfully deleted file: {file_path}")
    except PermissionError:
        print(f"Permission denied to delete file: {file_path} (File might be in use by another program)")
    except Exception as e:
        print(f"Error deleting file {file_path}: {e}")
"""

'\ndef delete_local_file(file_path):\n    try:\n        file_path.unlink(missing_ok=True)  # missing_ok=True prevents exception if file doesn\'t exist\n        print(f"Successfully deleted file: {file_path}")\n    except PermissionError:\n        print(f"Permission denied to delete file: {file_path} (File might be in use by another program)")\n    except Exception as e:\n        print(f"Error deleting file {file_path}: {e}")\n'

### """Delete encoded_validation_dataset.zip in NO_PUSH if present"""

IT'S IMPOSSIBLE IN COLAB!
When you run code in Colab, it executes on Google's servers (in the cloud), not on your local PC - unless you specifically set up a connection between them.

files.download() in Colab sends files to your local PC, but the opposite operation - deleting local files from Colab - isn't directly possible because of security restrictions in web browsers.

Why you can download but not delete locally?
- Downloads are initiated by your browser (you choose where to save)
- File deletion would require access to your local file system, which browsers strictly prohibit for security reasons. Colab isn't meant to manage local files.

**files.download()** is less stable and reliable than Google Drive

In [ ]:
"""
delete_local_file(encoded_validation_dataset_zip_NO_PUSH_path)
"""

'\ndelete_local_file(encoded_validation_dataset_zip_NO_PUSH_path)\n'

### """Mount Google Drive"""
drive.mount('/content/drive') gives access to your Google Drive under the path /content/drive/MyDrive/

### """Google Cloud Authentication"""

In [ ]:
"""
auth.authenticate_user()  # user = c.lepere@ictjob.be
"""

'\nauth.authenticate_user()  # user = c.lepere@ictjob.be\n'

In [ ]:
"""
mountpoint = '/content/drive'

# Check whether the directory is not empty and remove its contents
if os.path.isdir(mountpoint) and os.listdir(mountpoint):
    print(f"Mountpoint '{mountpoint}' is not empty. Removing contents...")
    for item in os.listdir(mountpoint):
        item_path = os.path.join(mountpoint, item)
        if os.path.isfile(item_path):
            os.remove(item_path)
            print(f"Removed file: {item_path}")
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
            print(f"Removed directory: {item_path}")
    print(f"Contents of '{mountpoint}'removed successfully.")
else:
    print(f"Mountpoint '{mountpoint}' is empty or does not exist.")

# Now proceed with mounting the drive
drive.mount(mountpoint)
# os.makedirs is required at both the source and destination but after drive.mount
os.makedirs(encoded_validation_dataset_drive_path, exist_ok=True)
"""

'\nmountpoint = \'/content/drive\'\n\n# Check whether the directory is not empty and remove its contents\nif os.path.isdir(mountpoint) and os.listdir(mountpoint):\n    print(f"Mountpoint \'{mountpoint}\' is not empty. Removing contents...")\n    for item in os.listdir(mountpoint):\n        item_path = os.path.join(mountpoint, item)\n        if os.path.isfile(item_path):\n            os.remove(item_path)\n            print(f"Removed file: {item_path}")\n        elif os.path.isdir(item_path):\n            shutil.rmtree(item_path)\n            print(f"Removed directory: {item_path}")\n    print(f"Contents of \'{mountpoint}\'removed successfully.")\nelse:\n    print(f"Mountpoint \'{mountpoint}\' is empty or does not exist.")\n\n# Now proceed with mounting the drive\ndrive.mount(mountpoint)\n# os.makedirs is required at both the source and destination but after drive.mount\nos.makedirs(encoded_validation_dataset_drive_path, exist_ok=True)\n'

### Logging

In [ ]:
# Is /content the current directory?
print(f"Current directory: {os.getcwd()}")

# Ensure the logs directory exists
os.makedirs('/content/logs', exist_ok=True)

# Create handlers for both file and console output
file_handler = logging.FileHandler('/content/logs/training.log')
file_handler.setLevel(logging.INFO)                               # Log level for file

#console_handler = logging.StreamHandler()
#console_handler.setLevel(logging.INFO)                            # Log level for console

# Create a formatter and attach it to both handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
#console_handler.setFormatter(formatter)

# Add the handlers to the root logger
logger = logging.getLogger()
#logger.setLevel(logging.INFO)
logger.addHandler(file_handler)
#logger.addHandler(console_handler)

#transformers_logging.set_verbosity_error()  # Only show errors, suppress info

# Remove all existing handlers of type StreamHandler
#for handler in logger.handlers[:]:
#    if isinstance(handler, logging.StreamHandler):
#        logger.removeHandler(handler)

# Verify by printing current handlers
#print("handlers", logger.handlers)  # Should only show FileHandler

# Test logging to ensure both file and console work
logging.info("Testing log before training starts.")


Current directory: /content


#### Hugging Face Hub (HF Hub) authenticate

In [ ]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')
api                    = HfApi()

if ACCESS_TO_HF_HUB:
    login(token=hf_token)

    # Check
    user = whoami(token=hf_token)
    assert user['name'] == 'claudelepere', f"{user['name']} is not claudelepere"
    print(f"user: {user}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


user: {'type': 'user', 'id': '66ec3d5f61228b02f8780beb', 'name': 'claudelepere', 'fullname': 'Claude Lepère', 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/66ec3d5f61228b02f8780beb/gvnf9pvm2KvE90ETMUQo3.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'jobs_token', 'role': 'fineGrained', 'createdAt': '2025-01-04T17:44:35.493Z', 'fineGrained': {'canReadGatedRepos': False, 'global': [], 'scoped': [{'entity': {'_id': '66ec3d5f61228b02f8780beb', 'type': 'user', 'name': 'claudelepere'}, 'permissions': ['repo.content.read', 'repo.write']}]}}}}


In [ ]:
if ACCESS_TO_HF_HUB:
    model_repoUrl   = create_repo(repo_id=repo_id, repo_type="model",   private=True, exist_ok=True)
    dataset_repoUrl = create_repo(repo_id=repo_id, repo_type="dataset", private=True, exist_ok=True) # must be created if you want it

    #create_branch(repo_id=repo_id, repo_type="model",   branch=branch, exist_ok=True)
    #create_branch(repo_id=repo_id, repo_type="dataset", branch=branch, exist_ok=True)

    print(f"Model Repo Url: {model_repoUrl} created successfully as a private repo")
    print(f"Dataset Repo Url: {dataset_repoUrl} created successfully as a private repo")



Model Repo Url: https://huggingface.co/claudelepere/jobs_EN_791112_50000_all_2025-07-04 created successfully as a private repo
Dataset Repo Url: https://huggingface.co/datasets/claudelepere/jobs_EN_791112_50000_all_2025-07-04 created successfully as a private repo


### Save locally, then upload model card to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:
    name            = "model_card"
    model_card_path = "model_card.md"

    with open(model_card_path, "w") as f:
        f.write(model_card)

    api.upload_file(
        path_or_fileobj = model_card_path,
        path_in_repo    = 'README.md',
        repo_id         = repo_id,
        commit_message  = f"{name}-{timestamp}"
    )

    print(f"{name} successfully uploaded to HF Hub as {model_card_path}")

No files have been modified since last commit. Skipping to prevent empty commit.


model_card successfully uploaded to HF Hub as model_card.md


In [ ]:
"""
# Check the Python version
print(sys.version)
print()

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
print()

# Check system information
!cat /etc/os-release
!uname -m
print()

# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#print()

# Check RAM
!free -h
print()

# Check disk space
!df -h
print()

# Get environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")
"""
!python -V
print(f"currentdir: {os.getcwd()}")

Python 3.11.13
currentdir: /content


### NO_PUSH => dataset_EN_791112_0_50000_all.zip

In [ ]:
# 📌 STEP 2: Upload your zip file to Colab
from google.colab import files

# Upload via browser
uploaded = files.upload()  # ← Select your .zip file

# ✅ Change this if your file has a different name
zip_filename = "dataset_EN_791112_0_50000_all.zip"

Saving dataset_EN_791112_0_50000_all.zip to dataset_EN_791112_0_50000_all.zip


In [ ]:
# 📌 STEP 3: Move and unzip into /content/uploaded_files
!mkdir -p /content/uploaded_files
!mv $zip_filename /content/uploaded_files/
!unzip -o /content/uploaded_files/$zip_filename -d /content/uploaded_files/

Archive:  /content/uploaded_files/dataset_EN_791112_0_50000_all.zip
  inflating: /content/uploaded_files/dataset_EN_791112_0_50000_all/dataset_dict.json  
  inflating: /content/uploaded_files/dataset_EN_791112_0_50000_all/test/data-00000-of-00001.arrow  
  inflating: /content/uploaded_files/dataset_EN_791112_0_50000_all/test/dataset_info.json  
  inflating: /content/uploaded_files/dataset_EN_791112_0_50000_all/test/state.json  
  inflating: /content/uploaded_files/dataset_EN_791112_0_50000_all/train/data-00000-of-00001.arrow  
  inflating: /content/uploaded_files/dataset_EN_791112_0_50000_all/train/dataset_info.json  
  inflating: /content/uploaded_files/dataset_EN_791112_0_50000_all/train/state.json  
  inflating: /content/uploaded_files/dataset_EN_791112_0_50000_all/validation/data-00000-of-00001.arrow  
  inflating: /content/uploaded_files/dataset_EN_791112_0_50000_all/validation/dataset_info.json  
  inflating: /content/uploaded_files/dataset_EN_791112_0_50000_all/validation/state.

In [ ]:
dataset_dict = load_datasetdict_from_zip_to_import.setup_and_load_dataset(
    subdir_name  = "uploaded_files",
    zip_filename = "dataset_EN_791112_0_50000_all.zip"
)

# ✅ DatasetDict ready
print(f"dataset_dict: {type(dataset_dict)} shape={dataset_dict.shape}\n{dataset_dict}")
print(f"dataset_dict.keys(): {dataset_dict.keys()}")
print(f"dataset_dict['train']:      {type(dataset_dict['train'])} shape={dataset_dict['train'].shape}")
print(f"dataset_dict['validation']: {type(dataset_dict['validation'])} shape={dataset_dict['validation'].shape}")
print(f"dataset_dict['test']:       {type(dataset_dict['test'])} shape={dataset_dict['test'].shape}")

#raise Exception("AAAA")


🔍 Looking for dataset at: /content/uploaded_files/dataset_EN_791112_0_50000_all
📂 Loading DatasetDict from: /content/uploaded_files/dataset_EN_791112_0_50000_all
✅ Loaded DatasetDict dataset_dict.
dataset_dict: <class 'datasets.dataset_dict.DatasetDict'> shape={'train': (40000, 149), 'validation': (5000, 149), 'test': (5000, 149)}
DatasetDict({
    train: Dataset({
        features: ['id', 'text', '142', '143', '146', '147', '149', '150', '152', '154', '155', '156', '158', '160', '162', '165', '167', '169', '170', '171', '173', '174', '175', '176', '356', '360', '361', '362', '364', '370', '371', '373', '375', '376', '394', '408', '667', '686', '687', '689', '756', '758', '760', '761', '900', '901', '902', '903', '905', '906', '907', '908', '909', '911', '912', '913', '914', '916', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '283', '284', '285', '286', '287', '288', '289', '291', '292', '293', '297', '298', '299', '300', '301

### """W&B initialization (not used now)"""

In [ ]:
"""
run_name = f"EN_{skills}_{all_rows_low}_{all_rows_high}_ml{max_length}_ep{epochs}_lr{learning_rate}_th{threshold}_at{attention_window}_{fp}"

if 'gradient_accumulation_steps' not in globals():
  run_name = f"{run_name}_ba{batch_size}"
else:
  run_name = f"{run_name}_ba{batch_size}x{gradient_accumulation_steps}"

print(f"run_name: {run_name}")
"""

'\nrun_name = f"EN_{skills}_{all_rows_low}_{all_rows_high}_ml{max_length}_ep{epochs}_lr{learning_rate}_th{threshold}_at{attention_window}_{fp}"\n\nif \'gradient_accumulation_steps\' not in globals():\n  run_name = f"{run_name}_ba{batch_size}"\nelse:\n  run_name = f"{run_name}_ba{batch_size}x{gradient_accumulation_steps}"\n\nprint(f"run_name: {run_name}")\n'

In [ ]:
"""
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ
wandb_api_key               = os.environ.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)
"""

'\nos.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ\nwandb_api_key               = os.environ.get(\'WANDB_API_KEY\')\nwandb.login(key=wandb_api_key)\n'

In [ ]:
"""
try:
  wandb.init(
      project = "skill_classification",
      name    = run_name,
      entity  = "claudelepere-c-cile-cy",
      config  = {
          "learning_rate": learning_rate,
          "epochs"       : epochs,
          "batch_size"   : batch_size
      }
  )
except wandb.CommError as err:
  print(f"CommError: {err}")
except Exception as exc:
  print(f"Exception: {exc}")
"""

'\ntry:\n  wandb.init(\n      project = "skill_classification",\n      name    = run_name,\n      entity  = "claudelepere-c-cile-cy",\n      config  = {\n          "learning_rate": learning_rate,\n          "epochs"       : epochs,\n          "batch_size"   : batch_size\n      }\n  )\nexcept wandb.CommError as err:\n  print(f"CommError: {err}")\nexcept Exception as exc:\n  print(f"Exception: {exc}")\n'

In [ ]:
example = dataset_dict['train'][0]

print(f"dataset_dict['train'][0]: {type(example)} {example.keys()}\n{example}")


dataset_dict['train'][0]: <class 'dict'> dict_keys(['id', 'text', '142', '143', '146', '147', '149', '150', '152', '154', '155', '156', '158', '160', '162', '165', '167', '169', '170', '171', '173', '174', '175', '176', '356', '360', '361', '362', '364', '370', '371', '373', '375', '376', '394', '408', '667', '686', '687', '689', '756', '758', '760', '761', '900', '901', '902', '903', '905', '906', '907', '908', '909', '911', '912', '913', '914', '916', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '283', '284', '285', '286', '287', '288', '289', '291', '292', '293', '297', '298', '299', '300', '301', '302', '303', '304', '358', '359', '365', '367', '369', '379', '382', '385', '409', '610', '611', '613', '614', '615', '618', '619', '637', '668', '670', '672', '673', '676', '677', '678', '681', '682', '683', '810', '811', '812', '813', '814', '815', '816', '817', '818', '819', '820', '821', '822', '823', '824', '825', '826', '82

### Create labels (list of str), id2label (int => str) and label2id (str => int).
* dataset 7_1000_125_125  ,  48 labels
* dataset 7_128_18_54     ,  42 labels
* dataset 8910_1087_68_204, 206 labels
* dataset 11_1000         ,   6 labels

In [ ]:
labels     = [label for label in dataset_dict['train'].features.keys() if label not in ['id', 'text']]
num_labels = len(labels)
id2label   = {idx: label for idx, label in enumerate(labels)}
label2id   = {label: idx for idx, label in enumerate(labels)}

print(f"labels    : {type(labels)}   len={len(labels)}\n{labels}")
print(f"num_labels: {num_labels}")
print(f"id2label  : {type(id2label)} len={len(id2label)}\n{id2label}")
print(f"label2id  : {type(label2id)} len={len(label2id)}\n{label2id}")

labels_key_types_set   = {type(key) for key in labels}
id2label_key_types_set = {type(key) for key in id2label}
label2id_key_types_set = {type(key) for key in label2id}
print(f"labels_key_types_set  : len={len(labels_key_types_set)}   {labels_key_types_set}")    # labels_key_types_set  : len=1   {<class 'str'>}
print(f"label2id_key_types_set: len={len(label2id_key_types_set)} {label2id_key_types_set}")  # label2id_key_types_set: len=1 {<class 'str'>}
print(f"id2label_key_types_set: len={len(id2label_key_types_set)} {id2label_key_types_set}")  # id2label_key_types_set: len=1 {<class 'int'>}

#raise Exception("BBBB")


labels    : <class 'list'>   len=147
['142', '143', '146', '147', '149', '150', '152', '154', '155', '156', '158', '160', '162', '165', '167', '169', '170', '171', '173', '174', '175', '176', '356', '360', '361', '362', '364', '370', '371', '373', '375', '376', '394', '408', '667', '686', '687', '689', '756', '758', '760', '761', '900', '901', '902', '903', '905', '906', '907', '908', '909', '911', '912', '913', '914', '916', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '283', '284', '285', '286', '287', '288', '289', '291', '292', '293', '297', '298', '299', '300', '301', '302', '303', '304', '358', '359', '365', '367', '369', '379', '382', '385', '409', '610', '611', '613', '614', '615', '618', '619', '637', '668', '670', '672', '673', '676', '677', '678', '681', '682', '683', '810', '811', '812', '813', '814', '815', '816', '817', '818', '819', '820', '821', '822', '823', '824', '825', '826', '827', '828', '829', '830', '13

### labels and id2label => HF Hub

WARNING:
- labels_dataset = Dataset.from_dict({'labels': labels, 'id2label': id2label}) creates a Dataset, not a DatasetDict.
- Storing id2label, a Python dictionary, as columns is not typical and can cause serialization problems or odd behavior.
- <mark>Due to the JSON conversion required for uploading to HF Hub, id2label (item of the dict labels) in HF Hub is a dict of str: str and it is also a dict of str: str when downloaded (item of the dict labels),<br>
unlike labels that was and remains a list of str</mark>

In [ ]:
"""
# Not good to convert to a Dataset to upload in datasets repo, good to upload a dict as a json file.
labels_dataset = Dataset.from_dict({
    'labels': labels,
    'id2label': id2label,
})
labels_dataset.push_to_hub(repo_id, 'labels')
"""

if ACCESS_TO_HF_HUB:
    filename = "labels.json"
    with open(filename, "w") as f:
        json.dump({
            'labels': labels,
            'id2label': id2label
        }, f)

    labels_path = api.upload_file(
        path_or_fileobj = filename,
        path_in_repo    = f"labels/{filename}",
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{filename}-{timestamp}"
    )
    print(f"labels_path: {type(labels_path)} len={len(labels_path)}\n{labels_path}")

    labels_path = api.hf_hub_download(
        repo_id   = repo_id,
        repo_type = 'model',
        filename  = f"labels/{filename}"
    )
    print(f"labels_path: {type(labels_path)} len={len(labels_path)}\n{labels_path}")

    with open(labels_path, "r") as f:
        labels_dict = json.load(f)
    print(f"labels_dict['labels']  : {type(labels_dict['labels'])}   len={len(labels_dict['labels'])}\n{labels_dict['labels']}")
    print(f"labels_dict['id2label']: {type(labels_dict['id2label'])} len={len(labels_dict['id2label'])}\n{labels_dict['id2label']}")

    #raise Exception("CCCC")


No files have been modified since last commit. Skipping to prevent empty commit.


labels_path: <class 'huggingface_hub.hf_api.CommitInfo'> len=100
https://huggingface.co/claudelepere/jobs_EN_791112_50000_all_2025-07-04/blob/main/labels/labels.json


labels.json:   0%|          | 0.00/3.00k [00:00<?, ?B/s]

labels_path: <class 'str'> len=156
/root/.cache/huggingface/hub/models--claudelepere--jobs_EN_791112_50000_all_2025-07-04/snapshots/dde243e4f598df3d3fc76ca99de5bf2d6c11412b/labels/labels.json
labels_dict['labels']  : <class 'list'>   len=147
['142', '143', '146', '147', '149', '150', '152', '154', '155', '156', '158', '160', '162', '165', '167', '169', '170', '171', '173', '174', '175', '176', '356', '360', '361', '362', '364', '370', '371', '373', '375', '376', '394', '408', '667', '686', '687', '689', '756', '758', '760', '761', '900', '901', '902', '903', '905', '906', '907', '908', '909', '911', '912', '913', '914', '916', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '283', '284', '285', '286', '287', '288', '289', '291', '292', '293', '297', '298', '299', '300', '301', '302', '303', '304', '358', '359', '365', '367', '369', '379', '382', '385', '409', '610', '611', '613', '614', '615', '618', '619', '637', '668', '670', '

### Load the pretrained tokenizer and the model

In [ ]:
model_name = "allenai/longformer-base-4096"


In [ ]:
# Increase the download timeout for Hugging Face Hub operations
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '60'
tokenizer                             = LongformerTokenizerFast.from_pretrained(model_name)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
# Check the type of the keys
print(f"id2label_key_types_set: len={len(id2label_key_types_set)} {id2label_key_types_set}")  # id2label_key_types_set: len=1 {<class 'int'>}
print(f"label2id_key_types_set: len={len(label2id_key_types_set)} {label2id_key_types_set}")  # label2id_key_types_set: len=1 {<class 'str'>}

model = LongformerForSequenceClassification.from_pretrained(
    model_name,
    num_labels   = num_labels,
    id2label     = id2label,
    label2id     = label2id,
    problem_type = 'multi_label_classification'
)

# Configure attention window size
model.config.attention_window = attention_window


id2label_key_types_set: len=1 {<class 'int'>}
label2id_key_types_set: len=1 {<class 'str'>}


pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Optimizer Initialization
- optimizer: will be used in the training loop to update the model's parameters.
- torch.optim.AdamW: This specifies the type of optimizer being used. AdamW is a popular optimization algorithm in deep learning. It's a variation of the Adam optimizer that includes "weight decay" (the "W"). Weight decay is a regularization technique that helps prevent the model from overfitting to the training data by penalizing large weights.
- model.parameters(): This is the first argument passed to the AdamW optimizer. It's a method provided by PyTorch Module (which your model object inherits from) that returns an iterator over all the trainable parameters (like weights and biases) of the model. The optimizer needs to know which parameters it should update during training.
- lr=learning_rate: This is the second argument, lr stands for "learning rate". It's a hyperparameter that controls the step size at each iteration while moving towards a minimum of the loss function. A smaller learning rate means smaller steps, which can lead to more stable training but might take longer. A larger learning rate means larger steps, which can speed up training but might overshoot the minimum or lead to unstable training.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


### Tokenize ('input_ids' and 'attention_mask'), add 'global_attention_mask' (for Longformer), add 'labels'

In [ ]:
def preprocess_data(examples, indices):
  # Step 1: Extract text and tokenize
  text = examples['text']             # Batch of texts
  encoding = tokenizer(
      text,
      truncation     = True,
      padding        = 'max_length',
      max_length     = max_length,
      return_tensors = 'pt'           # Return PyTorch tensors
  )

  # Step 2: Create and add the global attention mask
  global_attention_mask             = torch.zeros_like(encoding['input_ids'])  # Initialize global attention mask with zeros (same shape as input_ids)
  global_attention_mask[:, 0]       = 1                                        # Set global attention on the first token ([CLS], token ID=0) in each sequence
  encoding['global_attention_mask'] = global_attention_mask                    # Add the global_attention_mask to the batch

  # Step 3: Create and populate the label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)   # Create an empty label matrix
  #print(f"created labels_matrix: {type(labels_matrix)} shape={labels_matrix.shape}\n{labels_matrix}")

  #---------Populate label matrix
  for idx, label in enumerate(labels):
    # idx:0 label:142 idx:1 label:143 ... idx:145 label:352 idx:146 label:353
    #print(f"idx:{idx} label:{label}")
    if label in examples:
        labels_matrix[:, idx] = torch.tensor(
            [1.0 if val else 0.0 for val in examples[label]],
            dtype=torch.float32
        )
  # labels_matrix: <class 'torch.Tensor'> shape=torch.Size([1000, 147])
  # tensor([[0., 0., 0.,  ..., 0., 1., 0.],
  #      ...,
  #      [0., 0., 0.,  ..., 1., 0., 0.]])
  #print(f"labels_matrix: {type(labels_matrix)} shape={labels_matrix.shape}\n{labels_matrix}")

  encoding['labels'] = labels_matrix                                           # Add labels to the encoding
  # encoding['labels']: <class 'torch.Tensor'> shape=torch.Size([1000, 147])
  #print(f"encoding['labels']: {type(encoding['labels'])} shape={encoding['labels'].shape}")

  # encoding: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
  #   'input_ids': tensor([[
  #   'attention_mask': tensor([[
  #   'global_attention_mask': tensor([[
  #   'labels': tensor([[
  #print(f"1 preprocess_data call: encoding: {type(encoding)} {encoding.keys()}")

  return encoding


### Create encoded_dataset (datasets.dataset_dict.DatasetDict) = 3 encoded datasets.arrow_dataset.Dataset, train, validation and test

In [ ]:
encoded_dataset = dataset_dict.map(
    preprocess_data,
    batched        = True,
    remove_columns = dataset_dict['train'].column_names,
    with_indices   = True
)

print(f"encoded_dataset: {type(encoded_dataset)} shape={encoded_dataset.shape}")

#raise Exception("DDDD")


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/597M [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

encoded_dataset: <class 'datasets.dataset_dict.DatasetDict'> shape={'train': (40000, 4), 'validation': (5000, 4), 'test': (5000, 4)}


### Set PyTorch format to ensures correctness and compatibility with PyTorch pipelines
The 3 Hugging Face Dataset are formatted as PyTorch Dataset.

In [ ]:
encoded_dataset.set_format('torch')

train_dataset      = encoded_dataset['train']
validation_dataset = encoded_dataset['validation']
test_dataset       = encoded_dataset['test']

print(f"train_dataset_tensor:                          {type(train_dataset)} {train_dataset.shape}\t{train_dataset.features}\n{train_dataset}")
print(f"train_dataset_tensor['input_ids']:             {type(train_dataset['input_ids'])}\tlen={len(train_dataset['input_ids'])}\tshape={train_dataset['input_ids'].shape}\n{train_dataset['input_ids']}")
print(f"train_dataset_tensor['attention_mask']:        {type(train_dataset['attention_mask'])}\tlen={len(train_dataset['attention_mask'])}\tshape={train_dataset['attention_mask'].shape}\n{train_dataset['attention_mask']}")
print(f"train_dataset_tensor['global_attention_mask']: {type(train_dataset['global_attention_mask'])}\tlen={len(train_dataset['global_attention_mask'])}\tshape={train_dataset['global_attention_mask'].shape}\n{train_dataset['global_attention_mask']}")
print(f"train_dataset_tensor['labels']:                {type(train_dataset['labels'])}\tlen={len(train_dataset['labels'])}\tshape={train_dataset['labels'].shape}\n{train_dataset['labels']}")

#raise Exception("EEEE")


train_dataset_tensor:                          <class 'datasets.arrow_dataset.Dataset'> (40000, 4)	{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'global_attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}
Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 40000
})
train_dataset_tensor['input_ids']:             <class 'torch.Tensor'>	len=40000	shape=torch.Size([40000, 1024])
tensor([[    0,  3721,   448,  ...,     1,     1,     1],
        [    0, 26624,   250,  ...,     1,     1,     1],
        [    0, 44549,  7820,  ...,     1,     1,     1],
        ...,
        [    0,  5216,  1729,  ...,     1,     1,     1],
        [    0,  7743,   219,  ...,     1,     1,     1],
        [    0,   510,  118

### """Save Encoded Validation Dataset to Google Drive"""

In [ ]:
"""
# Save to disk and zip
validation_dataset.save_to_disk(encoded_validation_dataset_tmp)
shutil.make_archive(encoded_validation_dataset, 'zip', encoded_validation_dataset_tmp)

# Save to Google Drive
!cp {encoded_validation_dataset_zip} {encoded_validation_dataset_file_path}

# Download to NO_PUSH
#files.download(encoded_validation_dataset_zip)

#print("encoded_validation_dataset_zip successfully downloaded")
#raise Exception("STOP HERE")
"""

'\n# Save to disk and zip\nvalidation_dataset.save_to_disk(encoded_validation_dataset_tmp)\nshutil.make_archive(encoded_validation_dataset, \'zip\', encoded_validation_dataset_tmp)\n\n# Save to Google Drive\n!cp {encoded_validation_dataset_zip} {encoded_validation_dataset_file_path}\n\n# Download to NO_PUSH\n#files.download(encoded_validation_dataset_zip)\n\n#print("encoded_validation_dataset_zip successfully downloaded")\n#raise Exception("STOP HERE")\n'

#### """Check retrieval of encoded_validation_dataset.zip"""

In [ ]:
"""
#uploaded = files.upload()

#with zipfile.ZipFile(encoded_validation_dataset_zip, 'r') as zip_ref:
#    zip_ref.extractall(encoded_validation_dataset)

#encoded_validation_datasetZZZ = load_from_disk(encoded_validation_dataset)
#print(f"encoded_validation_datasetZZZ: {type(encoded_validation_datasetZZZ)} {encoded_validation_datasetZZZ}")

check_file_path = f"{encoded_validation_dataset_drive_path}/check.zip"
print(f"check_file_path: {check_file_path}")
!cp {encoded_validation_dataset_file_path} {check_file_path}

with zipfile.ZipFile(check_file_path, "r") as zip_ref:
    zip_ref.extractall('encoded_validation_dataset')

encoded_validation_dataset_ZZZ = load_from_disk('encoded_validation_dataset')
print(f"encoded_validation_dataset_ZZZ: {type(encoded_validation_dataset_ZZZ)} {encoded_validation_dataset_ZZZ.shape}\n{encoded_validation_dataset_ZZZ}")
#raise Exception("STOP")
"""

'\n#uploaded = files.upload()\n\n#with zipfile.ZipFile(encoded_validation_dataset_zip, \'r\') as zip_ref:\n#    zip_ref.extractall(encoded_validation_dataset)\n\n#encoded_validation_datasetZZZ = load_from_disk(encoded_validation_dataset)\n#print(f"encoded_validation_datasetZZZ: {type(encoded_validation_datasetZZZ)} {encoded_validation_datasetZZZ}")\n\ncheck_file_path = f"{encoded_validation_dataset_drive_path}/check.zip"\nprint(f"check_file_path: {check_file_path}")\n!cp {encoded_validation_dataset_file_path} {check_file_path}\n\nwith zipfile.ZipFile(check_file_path, "r") as zip_ref:\n    zip_ref.extractall(\'encoded_validation_dataset\')\n\nencoded_validation_dataset_ZZZ = load_from_disk(\'encoded_validation_dataset\')\nprint(f"encoded_validation_dataset_ZZZ: {type(encoded_validation_dataset_ZZZ)} {encoded_validation_dataset_ZZZ.shape}\n{encoded_validation_dataset_ZZZ}")\n#raise Exception("STOP")\n'

### encoded_validation_dataset => HF Hub

In [ ]:
encoded_validation_datasetDict = DatasetDict({'validation': encoded_dataset['validation']})

encoded_validation_datasetDict.push_to_hub(
    repo_id,
    config_name = 'encoded_validation_dataset' # This will create the directory on the hub
)

# Check
# Load the dataset from the specified repo_id and config_name
# The config_name is handled internally by load_dataset when loading from the hub
# You don't need to specify config_name again here if you are loading
# the configuration that was created with that config_name during push.
dsD = load_dataset(repo_id)
print(f"dsD: {type(dsD)} len={len(dsD)}\n{dsD}")

ds = dsD['validation']
print(f"ds: {type(ds)} shape={ds.shape}\n{ds}")

#raise Exception("FFFF")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/516 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


validation-00000-of-00001.parquet:   0%|          | 0.00/3.77M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

dsD: <class 'datasets.dataset_dict.DatasetDict'> len=1
DatasetDict({
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
        num_rows: 5000
    })
})
ds: <class 'datasets.arrow_dataset.Dataset'> shape=(5000, 4)
Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 5000
})


### Truncated part

In [ ]:
def get_truncated_part(text):
  tokens = tokenizer(
      text,
      truncation                = True,
      padding                   = 'max_length',
      max_length                = max_length,
      return_overflowing_tokens = True,
      return_tensors            = None
  )
  print(f"tokens.keys(): {tokens.keys()}")

  # Get the truncated tokens
  truncated_ids = tokens["input_ids"][0]
  print(f"truncated_ids: {type(truncated_ids)} {truncated_ids}")
  #overflow_ids  = tokens["overflow_to_sample_mapping"][0]
  #print(f"overflow_ids: {type(overflow_ids)} {overflow_ids}")

  # Decode the tokens back to text
  truncated_text = tokenizer.decode(truncated_ids, skip_special_tokens=True)
  #overflow_text  = tokenizer.decode(overflow_ids, skip_special_tokens=True)

  print(f"original_text :\n{text}")
  print(f"truncated_text:\n{truncated_text}")
  #print(f"overflow_text:\n{overflow_text}")

  original_tokens  = tokenizer.tokenize(text)
  truncated_tokens = tokenizer.tokenize(truncated_text)
  #overflow_tokens  = tokenizer.tokenize(overflow_text)

  print(f"original_tokens count : {len(original_tokens)}")
  print(f"truncated_tokens count: {len(truncated_tokens)}")
  #print(f"overflow_tokens count: {len(overflow_tokens)}")


### Forward pass for debugging or verification purposes (cf. BERT_3_1 in Notion BERT database) on dataset_dict['train'][0]['text']: inputs, outputs, logits, probs

In [ ]:
example_text = dataset_dict['train'][0]['text']
#get_truncated_part(example_text)


#### inputs

In [ ]:
inputs = tokenizer(
    example_text,
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

"""
inputs:
<class 'transformers.tokenization_utils_base.BatchEncoding'>
len=2
KeysView({'input_ids': tensor([[   0, 3721,  448,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])})
{'input_ids': tensor([[   0, 3721,  448,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}
"""
print(f"inputs:\n{type(inputs)}\nlen={len(inputs)}\n{inputs.keys()}\n{inputs}")

#raise Exception("GGGG")


inputs:
<class 'transformers.tokenization_utils_base.BatchEncoding'>
len=2
KeysView({'input_ids': tensor([[   0, 3721,  448,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])})
{'input_ids': tensor([[   0, 3721,  448,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


#### outputs


In [ ]:
outputs = model(
    input_ids      = inputs.input_ids,
    attention_mask = inputs.attention_mask
)

"""
outputs:
<class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'>
len=1
keys=odict_keys(['logits'])
LongformerSequenceClassifierOutput(loss=None, logits=tensor([[...]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)
"""
print(f"outputs:\n{type(outputs)}\nlen={len(outputs)}\nkeys={outputs.keys()}\n{outputs}")


Initializing global attention on CLS token...


outputs:
<class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'>
len=1
keys=odict_keys(['logits'])
LongformerSequenceClassifierOutput(loss=None, logits=tensor([[-0.0729,  0.0516,  0.1939, -0.2698, -0.0461,  0.1721, -0.1055, -0.0200,
         -0.1358,  0.0102,  0.0311,  0.0110,  0.0036, -0.0273, -0.0022, -0.0460,
         -0.0227,  0.0828, -0.0904,  0.0665, -0.0507,  0.0329, -0.1567,  0.0755,
         -0.1414,  0.1150, -0.1445,  0.1064,  0.0152,  0.0081,  0.0397, -0.2244,
          0.0840, -0.1657,  0.1665, -0.1831, -0.0722,  0.1096,  0.1934,  0.0652,
         -0.0395, -0.1552, -0.2175,  0.1465,  0.0837, -0.1575, -0.0998, -0.2095,
          0.0793, -0.0699, -0.0313,  0.1942, -0.2015, -0.1045,  0.2160, -0.1192,
         -0.1191,  0.1659, -0.0611, -0.2051, -0.1336, -0.0645, -0.0204, -0.0469,
          0.0286,  0.0541, -0.1183,  0.0214,  0.2080,  0.2144,  0.0758,  0.1584,
         -0.1186, -0.1046, -0.0045,  0.0211,  0.0911,  0.0895,  0.0314, -0.0789

#### logits
logits are around 0: normal for a model thay is not yet trained

In [ ]:
# Logits (= raw model outputs)
logits = outputs.logits

"""
logits:
<class 'torch.Tensor'>
shape=torch.Size([1, 147])
tensor([[...]], grad_fn=<AddmmBackward0>)
"""
print(f"logits:\n{type(logits)}\nshape={logits.shape}\n{logits}")


logits:
<class 'torch.Tensor'>
shape=torch.Size([1, 147])
tensor([[-0.0729,  0.0516,  0.1939, -0.2698, -0.0461,  0.1721, -0.1055, -0.0200,
         -0.1358,  0.0102,  0.0311,  0.0110,  0.0036, -0.0273, -0.0022, -0.0460,
         -0.0227,  0.0828, -0.0904,  0.0665, -0.0507,  0.0329, -0.1567,  0.0755,
         -0.1414,  0.1150, -0.1445,  0.1064,  0.0152,  0.0081,  0.0397, -0.2244,
          0.0840, -0.1657,  0.1665, -0.1831, -0.0722,  0.1096,  0.1934,  0.0652,
         -0.0395, -0.1552, -0.2175,  0.1465,  0.0837, -0.1575, -0.0998, -0.2095,
          0.0793, -0.0699, -0.0313,  0.1942, -0.2015, -0.1045,  0.2160, -0.1192,
         -0.1191,  0.1659, -0.0611, -0.2051, -0.1336, -0.0645, -0.0204, -0.0469,
          0.0286,  0.0541, -0.1183,  0.0214,  0.2080,  0.2144,  0.0758,  0.1584,
         -0.1186, -0.1046, -0.0045,  0.0211,  0.0911,  0.0895,  0.0314, -0.0789,
          0.0423,  0.2931,  0.0504, -0.0443,  0.0200, -0.0446, -0.0051,  0.0474,
         -0.0871, -0.1412,  0.1469, -0.0005, -0.052

#### probs
probs are around 0.5 i.e. for a given label (class) the model considers both possibilites (present or not) equally likely: normal for a model that is not yet trained: logits around 0 => probs around 0.5



In [ ]:
# Convert logits to probabilities
sigmoid = torch.nn.Sigmoid()
probs   = sigmoid(logits)

"""
probs:
<class 'torch.Tensor'>\torch.Size([1, 147])
tensor([[0.5433, 0.4414, 0.5040, 0.5300, 0.5740, 0.5057, 0.4828, 0.5061, 0.5059, ...]], grad_fn=<SigmoidBackward0>)
"""
print(f"probs:\n{type(probs)}\{probs.shape}\n{probs}")

#raise Exception("HHHH")


probs:
<class 'torch.Tensor'>\torch.Size([1, 147])
tensor([[0.4818, 0.5129, 0.5483, 0.4330, 0.4885, 0.5429, 0.4737, 0.4950, 0.4661,
         0.5025, 0.5078, 0.5028, 0.5009, 0.4932, 0.4994, 0.4885, 0.4943, 0.5207,
         0.4774, 0.5166, 0.4873, 0.5082, 0.4609, 0.5189, 0.4647, 0.5287, 0.4639,
         0.5266, 0.5038, 0.5020, 0.5099, 0.4441, 0.5210, 0.4587, 0.5415, 0.4543,
         0.4820, 0.5274, 0.5482, 0.5163, 0.4901, 0.4613, 0.4458, 0.5366, 0.5209,
         0.4607, 0.4751, 0.4478, 0.5198, 0.4825, 0.4922, 0.5484, 0.4498, 0.4739,
         0.5538, 0.4702, 0.4703, 0.5414, 0.4847, 0.4489, 0.4666, 0.4839, 0.4949,
         0.4883, 0.5071, 0.5135, 0.4705, 0.5053, 0.5518, 0.5534, 0.5190, 0.5395,
         0.4704, 0.4739, 0.4989, 0.5053, 0.5228, 0.5224, 0.5078, 0.4803, 0.5106,
         0.5728, 0.5126, 0.4889, 0.5050, 0.4888, 0.4987, 0.5119, 0.4782, 0.4648,
         0.5367, 0.4999, 0.4869, 0.5723, 0.4348, 0.5031, 0.5303, 0.4416, 0.5415,
         0.5199, 0.4903, 0.4742, 0.5139, 0.4889, 0.4673, 0

### firstly, example_text, then, example

In [ ]:
example = encoded_dataset['train'][0]
for k, v in example.items():
    if isinstance(v, np.ndarray):
        example[k] = v.tolist() # or v.astype(dtype) depending on your desired dtype


In [ ]:
print(f"example: {type(example)}\nlen={len(example)}\n{example.keys()}\n{example}")
print()
#print(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
#print(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
#print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")


example: <class 'dict'>
len=4
dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
{'input_ids': tensor([   0, 3721,  448,  ...,    1,    1,    1]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'global_attention_mask': tensor([1, 0, 0,  ..., 0, 0, 0]), 'labels': tensor([0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
        0., 1., 0.])}



In [ ]:
tokenizer.decode(example['input_ids'])


'<s>ELMOS - Oracle DBA Oracle, DBA , Clustering  ELMOS Database administration Definition + implementation new database objects Knowledge of clusters Backup & Recovery Monitoring Performance & Tuning Capacity planning Security Datafixes on the production environment Refresh database environments on demand Technical and functional consultancy RDBMS - Release Migration (Installation + Patches) Oracle SUBDOMAINS Databases: Oracle V8i, V9i, V10g (AIX, Solaris, Linux)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [ ]:
labels_list = [id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]
print(f"labels_list:\n{type(labels_list)}\nlen={len(labels_list)}\n{labels_list}")

labels_list_key_types = {type(label) for label in labels_list}
print(f"labels_list_key_types: {labels_list_key_types}")

#raise Exception("IIII")


labels_list:
<class 'list'>
len=11
['154', '155', '266', '284', '291', '299', '637', '137', '138', '139', '352']
labels_list_key_types: {<class 'str'>}


In [ ]:
"""
input_ids[0]:              <class 'torch.Tensor'>	torch.Size([1024])
attention_mask[0]:         <class 'torch.Tensor'>	torch.Size([1024])
global_attention_mask[0]:  <class 'torch.Tensor'>	torch.Size([1024])
labels[0]:                 <class 'torch.Tensor'>	torch.Size([147])
"""
print(f"input_ids[0]:              {type(encoded_dataset['train']['input_ids'][0])}\t{encoded_dataset['train']['input_ids'][0].shape}\tlen={len(encoded_dataset['train']['input_ids'][0])}")
print(f"attention_mask[0]:         {type(encoded_dataset['train']['attention_mask'][0])}\t{encoded_dataset['train']['attention_mask'][0].shape}\tlen={len(encoded_dataset['train']['attention_mask'][0])}")
print(f"global_attention_mask[0]:  {type(encoded_dataset['train']['global_attention_mask'][0])}\t{encoded_dataset['train']['global_attention_mask'][0].shape}\tlen={len(encoded_dataset['train']['global_attention_mask'][0])}")
print(f"labels[0]:                 {type(encoded_dataset['train'][0]['labels'])}\t{encoded_dataset['train'][0]['labels'].shape}\tlen={len(encoded_dataset['train'][0]['labels'])}")


input_ids[0]:              <class 'torch.Tensor'>	torch.Size([1024])	len=1024
attention_mask[0]:         <class 'torch.Tensor'>	torch.Size([1024])	len=1024
global_attention_mask[0]:  <class 'torch.Tensor'>	torch.Size([1024])	len=1024
labels[0]:                 <class 'torch.Tensor'>	torch.Size([147])	len=147


### Explain the following instruction
The selected code snippet performs a forward pass through your LongformerForSequenceClassification model with a single example from your training dataset.

Here's a breakdown:

outputs = model(...): This line calls your model object (which is an instance of LongformerForSequenceClassification) and passes input data to it. This is the core of the model's inference process.
input_ids = encoded_dataset['train']['input_ids'][0].unsqueeze(0):
encoded_dataset['train']['input_ids'][0]: This accesses the input_ids tensor for the first example ([0]) in your training dataset ('train').
.unsqueeze(0): Models typically expect inputs in batches, even if there's only one example. unsqueeze(0) adds a new dimension at the beginning of the tensor, effectively creating a batch of size 1.
attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0): Similar to input_ids, this accesses the attention_mask for the first example and adds a batch dimension. The attention mask tells the model which tokens are actual input and which are padding.
labels = encoded_dataset['train'][0]['labels'].unsqueeze(0): This accesses the ground truth labels for the first example and adds a batch dimension. Providing labels during the forward pass allows the model to calculate the loss, which is useful for debugging or understanding how the model is performing on a specific example.
In essence, this code is feeding one data point (the first example from the training set, including its text tokenized as input_ids and attention_mask, and its corresponding labels) into the model to get the model's output (outputs). Since labels are provided, the outputs object will likely include the calculated loss for this single example.


Is it required? I have cells above without unsqueeze.
That's a great observation! You're right, you didn't see .unsqueeze(0) in the cells where you were processing the entire dataset_dict or using the trainer.

The reason it's used in the selected cell (VN-sEDggwcJ4) is because you are explicitly taking a single example from the encoded_dataset['train'] ([0]) and passing it directly to the model.

Here's the difference:

When processing the whole dataset or using the Trainer: The Hugging Face datasets library and the Trainer are designed to handle data in batches. When you iterate through a dataset or pass it to the Trainer, the library automatically groups the examples into batches and provides them to the model with the correct batch dimension already present. You don't need to manually add it.
When processing a single example manually: If you want to feed just one example to the model outside of the batching mechanism provided by datasets or Trainer (like in the selected cell for demonstration or debugging), you need to manually add the batch dimension using .unsqueeze(0). This is because most models are designed to process data in batches, even if the batch size is one. The model expects input tensors of shape (batch_size, sequence_length) or (batch_size, num_labels), not just (sequence_length) or (num_labels).
So, while unsqueeze(0) is not needed when the data is already batched (by datasets or Trainer), it is required when you are working with a single example tensor and feeding it directly to a model that expects batched input.



In [ ]:
outputs = model(
    input_ids      = encoded_dataset['train']['input_ids'][0].unsqueeze(0),
    attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
    labels         = encoded_dataset['train'][0]['labels'].unsqueeze(0)
)

"""
outputs:
<class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'>
odict_keys(['loss', 'logits'])
LongformerSequenceClassifierOutput(loss=tensor(0.6918, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 8.3523e-02, ..., -3.2362e-02]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)
"""
print(f"outputs:\n{type(outputs)}\n{outputs.keys()}\n{outputs}")

#raise Exception("JJJJ")


outputs:
<class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'>
odict_keys(['loss', 'logits'])
LongformerSequenceClassifierOutput(loss=tensor(0.6910, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0729,  0.0516,  0.1939, -0.2698, -0.0461,  0.1721, -0.1055, -0.0200,
         -0.1358,  0.0102,  0.0311,  0.0110,  0.0036, -0.0273, -0.0022, -0.0460,
         -0.0227,  0.0828, -0.0904,  0.0665, -0.0507,  0.0329, -0.1567,  0.0755,
         -0.1414,  0.1150, -0.1445,  0.1064,  0.0152,  0.0081,  0.0397, -0.2244,
          0.0840, -0.1657,  0.1665, -0.1831, -0.0722,  0.1096,  0.1934,  0.0652,
         -0.0395, -0.1552, -0.2175,  0.1465,  0.0837, -0.1575, -0.0998, -0.2095,
          0.0793, -0.0699, -0.0313,  0.1942, -0.2015, -0.1045,  0.2160, -0.1192,
         -0.1191,  0.1659, -0.0611, -0.2051, -0.1336, -0.0645, -0.0204, -0.0469,
          0.0286,  0.0541, -0.1183,  0.0214,  0.2080,  0.2144,  0.0758,  0.1584,
         -0.1186, -0.10

### Workflow

- 3 steps: training (train_dataset), evaluation (validation_dataset), test (test_dataset)
- 3 datasets: train, validation, test
- 3 Trainer functions: train, evaluate, predict


### Training step


In [ ]:
batch_size  = batch_size


#### Weighted loss function
**weight and pos_weight**:
- torch.nn.BCEWithLogitsLoss function is a commonly used loss function for binary classification problems, where model output is a probability value between 0 and 1. It combines a sigmoid activation function with a binary cross-entropy loss.
- For imbalanced datasets, where number of a class is significantly smaller than other, BCEWithLogitsLoss can be modified by adding a weight parameter to loss function.<br/>
BCEWithLogitsLoss also has a pos_weight parameter, which is a simpler way to specify weight for positive class (equivalent to weight parameter = [ 1, pos_weight], where weight for negative class = 1.<br>
Negative samples (0s) are not weighted explicitly because the loss function already balances them implicitly.
- pos_weights stands for positive weights in the BCEWithLogitsLoss function.<br/>
In multi-label classification, each label is a separate binary classification problem.<br/>
Each label has positive sample (1s) and negative samples (0s) in the dataset.
    - If a label is rare (fewer 1s), its weight will be higher -> encourages the model to predict it more often
    - If a label is common (many 1s), its weight will be lower -> prevents the model to overpredicting it

**Normalization**:
Without normalization, pos_weights might have huge variations across labels, that could destabilize training.
- Min-Max Scaling:
    - Rescales values between 0 and 1
    - Reduces large variations but keeps relative ranking
- Z-Score Normalization:
    - Centers values around 0 with a standard deviation of 1
    - Handles outliers better than min-max
- Sum-to-One Scaling:
    - Makes weights sum to 1, preventing extremely large values
- Recommended approach: try sum-to-one normalization first. If performance is unstable, test z-score.

#### Weighted BCEWithLogitsLoss
Assigns higher weights to rare labels using class weights.

In [ ]:
def class_weights(labels):
    print(f"labels: {type(labels)} len={len(labels)} shape={labels.shape}\n{labels}")

    num_samples, num_labels = labels.shape
    print(f"num_samples: {type(num_samples)} {num_samples}")
    print(f"num_labels:  {type(num_labels)}  {num_labels}")

    # class_counts = how many times each label appears (i.e. number of 1s per label)
    # (dim=0 means summing across all samples; equivalent to axis = 0 for Pandas DataFrame)
    class_counts = labels.sum(dim=0)
    print(f"class_counts: {type(class_counts)} len={len(class_counts)}\n{class_counts}")

    # pos_weights = negative samples (0s) per label / positive samples (1s) per label
    pos_weights = (num_samples-class_counts) / (class_counts + 1e-6)  # Avoid division by zero
    print(f"pos_weights: {type(pos_weights)} len={len(pos_weights)}\n{pos_weights}")

    # Normalization
    normalized_pos_weights_minmax = (pos_weights-pos_weights.min()) / (pos_weights.max()-pos_weights.min())
    print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

    normalized_pos_weights_zscore = (pos_weights-pos_weights.mean()) / pos_weights.std()
    print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

    normalized_pos_weights_sum1 = pos_weights / pos_weights.sum()
    print(f"normalized_pos_weights_sum1: {type(normalized_pos_weights_sum1)} {len(normalized_pos_weights_sum1)} {normalized_pos_weights_sum1}")

    #return pos_weights
    #return normalized_pos_weights_minmax
    #return normalized_pos_weights_zscore
    return normalized_pos_weights_sum1


In [ ]:
pos_weights = class_weights(encoded_dataset['train']['labels'])
bce_loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights.to(device))


labels: <class 'torch.Tensor'> len=40000 shape=torch.Size([40000, 147])
tensor([[0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])
num_samples: <class 'int'> 40000
num_labels:  <class 'int'>  147
class_counts: <class 'torch.Tensor'> len=147
tensor([1.3368e+04, 4.5500e+02, 2.7610e+03, 1.9470e+03, 8.1540e+03, 1.9070e+03,
        1.3600e+02, 9.8700e+02, 6.0400e+02, 2.9240e+03, 2.2700e+03, 5.8710e+03,
        2.1720e+03, 2.3080e+03, 4.5400e+02, 2.3640e+03, 5.1980e+03, 9.3100e+02,
        7.5300e+02, 9.3400e+02, 8.0700e+02, 3.1800e+02, 4.5240e+03, 2.8700e+02,
        1.0080e+03, 1.8020e+03, 1.9210e+03, 1.2000e+02, 3.0600e+02, 4.6300e+02,
        4.3100e+02, 4.7600e+02, 8.9400e+02, 2.0750e+03, 1.1900e+02, 3.3800e+02,
        2.2100e+02, 1.9280e+03, 1.6400e+02, 1.8800e+02, 3.6100e+02, 8.6400e+02,
    

#### Focal Loss
Reduces the impact of easy examples (majority class) and focuses on difficult cases.

- α (alpha): Adjusts class weighting (0.5 means equal weight). Higher α gives more weight to minority classes.
- γ (gamma): Controls how much hard-to-classify samples are emphasized. Higher γ reduces the influence of easy samples.

In [ ]:
class FocalLoss(Module):
    """
    Focal Loss implementation for handling class imbalance.
    """
    counter = 0

    def __init__(self, alpha=1.0, gamma=2.0, logits=True, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha     = alpha
        self.gamma     = gamma
        self.logits    = logits     # True if inputs are logits, False if probabilies
        self.reduction = reduction  # 'mean' or 'none'

        FocalLoss.counter += 1

    # inputs  = model's predictions: PyTorch tensor, shape=(batch_size, num_classes)
    # targets = ground truth labels: PyTorch tensor, shape=same as inputs shape
    def forward(self, inputs, targets):
        targets = targets.to(inputs.device)  # Ensure labels are on the same device

        #print(f"inputs: {type(inputs)} {inputs.shape}\ntargets: {type(targets)} {targets.shape}"
        # Here, we check if input is probability or logits
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')

        pt         = torch.exp(-BCE_loss)  # Probability of the correct class
        focal_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        return focal_loss.mean() if self.reduction == 'mean' else focal_loss

    def __repr__(self):
        return f"FocalLoss(alpha={self.alpha}, gamma={self.gamma}, logits={self.logits}, reduction={self.reduction})"

    def __str__(self):
        return self.__repr__()


In [ ]:
#focal_loss_fn = FocalLoss(alpha=0.5, gamma=3.0, logits=True, reduction='mean')
#focal_loss_fn = FocalLoss(alpha=0.25, gamma=4.0, logits=True, reduction='mean')
#focal_loss_fn = FocalLoss(alpha=0.5, gamma=4.0, logits=True, reduction='mean')
#focal_loss_fn = FocalLoss(alpha=0.625, gamma=4.0, logits=True, reduction='mean')

focal_loss_fn = FocalLoss(alpha=alpha, gamma=gamma, logits=True, reduction='mean')

print(f"focal_loss_fn: {type(focal_loss_fn)} {focal_loss_fn}")


focal_loss_fn: <class '__main__.FocalLoss'> FocalLoss(alpha=1.0, gamma=1.5, logits=True, reduction=mean)


In [ ]:
class LossLoggerCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        # Retrieve loss from logs
        logs       = kwargs.get("logs", {})
        epoch_loss = logs.get("logs", None)  # Loss from the Trainer logs

        if epoch_loss is not None:
            logging.info(f"Epoch {state.epoch:.0f} - Average Loss: {epoch_loss:.6f}")
            #print(f"Epoch {state.epoch:.0f} - Average Loss: {epoch_loss:.6f}", flush=True)
        else:
            logging.warning(f"Epoch {state.epoch:.0f} - No loss logged!")
            #print(f"Epoch {state.epoch:.0f} - No loss logged!", flush=True)


In [ ]:
class MetricsLoggerCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is None:
            metrics = kwargs.get("metrics")  # Ensure we get the metrics if passed in kwargs
        if metrics:                          # Check if metrics exist
            logging.info(f"Epoch {state.epoch:.0f} - "
                         f"Precision: {metrics.get('precision', float('nan')):.4f} - "
                         f"Recall: {metrics.get('recall', float('nan')):.4f} - "
                         f"F1: {metrics.get('f1', float('nan')):.4f}")


In [ ]:
class ProgressLoggerCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        logs       = kwargs.get("logs", {})  # Extract from logs
        epoch_loss = logs.get("loss", None)  # Get loss value

        if epoch_loss is not None:
            logging.info(f"Epoch {state.epoch:.0f} - Average Loss: {epoch_loss:.6f}")
            #print(f"Epoch {state.epoch:.0f} - Average Loss: {epoch_loss:.6f}", flush=True)
        else:
            logging.warning(f"Epoch {state.epoch:.0f} - No loss logged!")
            #rint(f"Epoch {state.epoch:.0f} - No loss logged!", flush=True)

    def on_train_end(self, args, state, control, **kwargs):
        logging.info("Training Completed!")
        #print("=== Training Completed! ===", flush=True)


#### Training Metrics
  source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

#### UndefinedMetricWarning
Only one class is present in y_true. ROC AUC score is not defined in that case.

This warning typically arises when you're trying to calculate the ROC AUC score for a label where either all true values are 0 or all are 1 in a particular batch.

The ROC AUC score is calculated by comparing the true positive rate (TPR) against the false positive rate (FPR) at various thresholds. If a label is only ever predicted as 0 or 1, you cannot generate a meaningful ROC curve and thus the AUC is undefined.

AUC scores rely on the presence of both positive and negative samples for each label.

Solution:

- Check Label Distribution: Add a check at the start to see if either true_labels or preds for a particular label contain only one unique value (0 or 1). If so, for that label, either skip the ROC AUC calculation or set the ROC AUC to a default value (like 0 or NaN).
- Ignore the warning (not recommended):
- **Stratified sampling: While you did split into train/validation/test, the warning may indicate you didn't maintain the label balance during the splitting process. Stratified sampling would do this.**

#### TPR (True Positive Rate) and FPR (False Positive Rate)
TPR = Sensitivity = Recall = TP / (TP + FN)
- TPR close to 1: the model identifies most positives
- TPR close to 0: the model is missing many positives

FPR = FP / (FP + TN)
- FPR close to 1: the model produces many false alarms
- FPR close to 0: the model makes few false alarms

The ROC curve plots TPR (y-axis) vs. FPR (x-axis) at different thresholds.
A perfect model has:
- TPR = 1 (detect all positives)
- FPR = 0 (no false alarms)

The ideal ROC curve is a steep rise towards the top-left corner.

#### zero_division=0

- only for f1, precision and recall because they involve division where the denominator can be zero: some labels might never       be predicted (y_pred = 0 for all samples), or they might not appear in the true_labels (y_true = 0 for all samples)
- ROC AUC: works with probabilities and does not involve division by zero
- Precision-Recall AUC: also based on ranking, so no zero division issue
- Accuracy: just compares exact matches, so no zero division issue

In [ ]:
def multi_label_metrics(logits, true_labels, threshold=0.5, average='micro'):
    """
    Compute multi-label classification metrics.

    Params:
    - logits      (np.ndarray or torch.Tensor): raw, unnormalized model outputs, shape (n_samples, n_labels)
    - true_labels (np.ndarray)                : ground truth binary labels shape (n_samples, n_labels)
    - threshold   (float or np.ndarray)       : 0.5 (default)
    - average     (str)                       : 'micro' (default); no per-label scores i.e. not average=None

    Returns:
    - metrics (dict)
    """
    global multi_label_metrics_counter
    if multi_label_metrics_counter == 0:
        print(">>>>>>>>>>multi_label_metrics entered for the first time<<<<<<<<<<", flush=True)
    multi_label_metrics_counter += 1

    print(f"multi_label_metrics: logits: {type(logits)} shape={logits.shape}\n{logits[:5]}")                      # <class 'numpy.ndarray'> (12, 6)
    print(f"multi_label_metrics: true_labels: {type(true_labels)} shape={true_labels.shape}\n{true_labels[:5]}")  # <class 'numpy.ndarray'> (12, 6)
    # Handle both float and array thresholds
    if isinstance(threshold, (int, float)):
        print(f"multi_label_metrics: threshold: {type(threshold)} {threshold}")
    else:
        print(f"multi_label_metrics: threshold: {type(threshold)} shape={threshold.shape}\n{threshold[:5]}")

    # Ensure logits are a PyTorch tensor
    if isinstance(logits, np.ndarray):
        logits = torch.from_numpy(logits)
    print(f"logits: {type(logits)} shape={logits.shape}\n{logits}")                      # <class 'torch.Tensor'> torch.Size([12, 6])

  # Apply sigmoid to get probabilities; convert to NumPy safely:
  # - detach() remove the tensor from the computation graph, making it a regular tensor without gradients
  # - cpu() moves the tensor from the GPU to the CPU before converting to NumPy
    probs = torch.sigmoid(logits).detach().cpu().numpy()
    print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")
    # <class 'numpy.ndarray'> (12, 6)

    # Apply threshold to get binary predictions
    # NOTE: probs is a np.ndarray, NumPy can compare a np.ndarray to
    # - a scalar (float/int/NumPy scalar), another np.ndarray of the same shape,
    # - but not to a PyTorch tensor
    preds = (probs > threshold).astype(int)
    print(f"preds: {type(preds)} shape={preds.shape}\n{preds}")                          # <class 'numpy.ndarray'> (12, 6)

    # Compute core metrics
    # Use zero_division='warn' or 0 or 1 explicitly
    # zero_division='warn' is the default and recommended approach
    # zero_division=0 treats zero division as resulting in 0
    # zero_division=1 treats zero division as resulting in 1
    f1        = f1_score       (true_labels, preds, average=average, zero_division='warn')
    precision = precision_score(true_labels, preds, average=average, zero_division='warn')
    recall    = recall_score   (true_labels, preds, average=average, zero_division='warn')

    # To avoid computing AUC scores on labels that are always 0 or always 1;
    # identify valid labels (those with both 0s and 1s in 'y_true' i.e. those present in some but not all samples)
    valid_labels = np.where(
        (true_labels.sum(axis=0) > 0) &
        (true_labels.sum(axis=0) < true_labels.shape[0])
    )[0]
    print(f"valid_labels: {type(valid_labels)} len={len(valid_labels)}\n{valid_labels}")

    # Curve-based metrics (mean across valid labels). These are label-wise, then averaged over valid labels.
    if len(valid_labels) > 0:
        roc_auc = np.mean([
            roc_auc_score(true_labels[:, i], probs[:, i])
            for i in valid_labels
        ])
        pr_recall_auc = np.mean([
            average_precision_score(true_labels[:, i], probs[:, i])
            for i in valid_labels
        ])
    else:
        roc_auc       = np.nan
        pr_recall_auc = np.nan

    # Other useful metrics:
    # - subset accuracy: % of samples with all labels correctly predicted, exact match between all predicted and true labels per sample
    # - hamming loss   : % of misclassified labels (lower is better, good for imbalanced data)
    subset_acc = accuracy_score(true_labels, preds)
    hamming    = hamming_loss(true_labels, preds)

    return {
        'f1'                  : f1,
        'precision'           : precision,
        'recall'              : recall,
        'roc_auc'             : roc_auc,
        'precision_recall_auc': pr_recall_auc,
        'subset_accuracy'     : subset_acc,
        'hamming_loss'        : hamming
    }


In [ ]:
# Usage with HF Trainer
def compute_metrics(p: EvalPrediction):
    global compute_metrics_counter
    if compute_metrics_counter == 0:
        print(">>>>>>>>>>compute_metrics entered for the first time<<<<<<<<<<", flush=True)
    compute_metrics_counter += 1

    print(f"p.predictions: {type(p.predictions)} len={len(p.predictions)}\n{p.predictions[:5]}")
    # p.predictions: <class 'numpy.ndarray'> len=5000
    print(f"p.label_ids: {type(p.label_ids)} len={len(p.label_ids)}\n{p.label_ids[:5]}")
    # p.label_ids: <class 'numpy.ndarray'> len=5000
    if isinstance(threshold, (int, float)):
        print(f"threshold: {type(threshold)} {threshold}") # threshold: <class 'float'> 0.5
    else:
        print(f"threshold: {type(threshold)} shape={threshold.shape}\n{threshold[:5]}")


    preds  = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions

    return multi_label_metrics(preds, p.label_ids, threshold=threshold, average=training_average)


#### HF transformer Trainer and CustomTrainer
Abstracts the training loop.

In [ ]:
training_args = TrainingArguments(
    output_dir                  = './training_results',  # where model predictions and checkpoints will be written during training
    overwrite_output_dir        = True,
    save_steps                  = 500,
    save_total_limit            = 2,
    eval_strategy               = 'epoch',               # Evaluate once per epoch
    save_strategy               = 'epoch',               # Save checkpoints once per epoch
    learning_rate               = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    num_train_epochs            = epochs,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = 'f1',
    greater_is_better           = True,
    fp16                        = True,
    #run_name                    = run_name,
    report_to                   = 'none'                 # Disable wandb if not needed
)


In [ ]:
class CustomTrainer(Trainer):
    compute_loss_counter = 0

    def __init__(self, model, *args, loss_fn=None, **kwargs):
        super().__init__(model, *args, **kwargs)
        self.loss_fn = loss_fn
        print(f">>>>>>>>>>CustomTrainer initialized with loss_fn: {loss_fn}<<<<<<<<<<")

    """
    # No print in compute_loss because out of memory because prints are batch per batch
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):

    #print(f"inputs passed to compute_loss: {inputs.keys()}")
    #input_ids             = inputs['input_ids']                        # shape: batch_size, sequence_length
    #attention_mask        = inputs['attention_mask']                   # shape: batch_size, sequence_length
    #global_attention_mask = inputs.get('global_attention_mask', None)  # shape: batch_size, sequence_length; optional as LongFormer specific
    labels                = inputs.pop('labels', None)                 # shape: batch_size, num_labels; needed for loss computation, not required by the model

    #outputs = model(**inputs, global_attention_mask=global_attention_mask)  # Forward pass
    # Forward pass
    #outputs = model(
    #    input_ids             = input_ids,
    #    attention_mask        = attention_mask,
    #    global_attention_mask = global_attention_mask,
    #    labels                = labels
    #)
    outputs = model(**inputs, labels=labels)
    #print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")
    logits = outputs.logits  # shape: (batch_size, num_labels)

    # If labels are provided, compute loss
    if labels is not None:
      # Use the custom loss function if provided
      if self.loss_fn is not None:
        loss = self.loss_fn(logits, labels)  # Compute weighted loss
      else:
        # Default loss: BCEWithLogitsLoss
        loss_fn = BCEWithLogitsLoss()
        loss    = loss_fn(logits, labels)    # Compute loss
      return (loss, outputs) if return_outputs else loss

    # If no labels, return outputs only, for evaluation or prediction
    return outputs
    """
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        if CustomTrainer.compute_loss_counter == 0:
            print(f">>>>>>>>>>CustomTrainer: compute_loss entered for the first time<<<<<<<<<<", flush=True)
        CustomTrainer.compute_loss_counter += 1

        labels  = inputs.get('labels')
        outputs = model(**inputs)
        logits  = outputs.logits  # (batch_size, num_labels)

        if labels is not None:
            labels = labels.to(logits.device).float()  # Ensure same device

            if self.loss_fn is not None:
                loss = self.loss_fn(logits, labels)
                logging.info(f"Step Loss ({self.loss_fn.__class__.__name__}): {loss.item():.6f}")  # Log loss value
                #print(f"Epoch {self.state.epoch:.0f}, Step {self.state.global_step}: Loss ({self.loss_fn.__class__.__name__}): {loss.item():.6f}", flush=True)
                # Epoch 0, Step 0: Loss (FocalLoss): 0.251234
            else:
                loss_fn = BCEWithLogitsLoss()
                loss    = loss_fn(logits, labels)
                logging.info(f"Step Loss (BCEWithLogitsLoss): {loss.item():.6f}")                  # Log loss value
                #print(f"Epoch {self.state.epoch:.0f}, Step {self.state.global_step}: Loss (BCEWithLogitsLoss): {loss.item():.6f}", flush=True)

            return (loss, outputs) if return_outputs else loss

        return outputs

    def get_CustomTrainer_compute_loss_counter(self):
        return CustomTrainer.compute_loss_counter


In [ ]:
trainer = CustomTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = encoded_dataset["train"],
    eval_dataset    = encoded_dataset["validation"],
    compute_metrics = compute_metrics,
    loss_fn         = focal_loss_fn,  # bce_loss_fn or focal_loss_fn
    #callbacks       = [LossLoggerCallback(), MetricsLoggerCallback(), ProgressLoggerCallback()]  # Attach logging callbacks
)

#trainer = Trainer(
#    model           = model,
#    args            = training_args,
#    train_dataset   = encoded_dataset["train"],
#    eval_dataset    = encoded_dataset["validation"],
#    compute_metrics = compute_metrics,
#)

print(f"trainer: {type(trainer)} {trainer}")


>>>>>>>>>>CustomTrainer initialized with loss_fn: FocalLoss(alpha=1.0, gamma=1.5, logits=True, reduction=mean)<<<<<<<<<<
trainer: <class '__main__.CustomTrainer'> <__main__.CustomTrainer object at 0x7afe6bb4a190>


#### trainer.train

In [ ]:
trainer_train = trainer.train()

print(f"trainer_train: {type(trainer_train)} len={len(trainer_train)}\n{trainer_train}")
print()
print(f"trainer_train.metrics: {type(trainer_train.metrics)} len={len(trainer_train.metrics)}\n{json.dumps(trainer_train.metrics, indent=4)}")

print(f"after trainer.train(): multi_label_metrics_counter       : {multi_label_metrics_counter}")
print(f"after trainer.train(): compute_metrics_counter           : {compute_metrics_counter}")
print(f"after trainer.train(): compute_loss_counter              : {compute_loss_counter}")
print(f"after trainer.train(): CustomTrainer_compute_loss_counter: {trainer.get_CustomTrainer_compute_loss_counter()}")
print(f"after trainer.train(): FocalLoss.counter                 : {FocalLoss.counter}")

#raise Exception("KKKK")


>>>>>>>>>>CustomTrainer: compute_loss entered for the first time<<<<<<<<<<


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Subset Accuracy,Hamming Loss,Runtime,Samples Per Second,Steps Per Second
1,0.033600,0.030335,0.638272,0.813863,0.525002,0.818074,0.274966,0.044000,0.026472,76.117700,65.688000,8.211000


>>>>>>>>>>compute_metrics entered for the first time<<<<<<<<<<
p.predictions: <class 'numpy.ndarray'> len=5000
[[ 1.3535156  -1.3964844  -1.5722656  -2.0351562  -0.77001953 -2.0449219
  -2.4433594  -2.1835938  -2.2128906  -1.8867188  -2.1445312  -1.8974609
  -2.0371094  -2.2460938  -2.2148438  -1.5615234  -1.7792969  -2.1347656
  -2.2285156  -2.0976562  -2.3222656  -2.3652344  -1.8818359  -2.2714844
  -2.0527344  -1.5185547  -1.7392578  -2.5136719  -2.0117188  -2.3867188
  -2.2480469  -2.3828125  -2.2890625  -2.2421875  -2.6054688  -1.6767578
  -2.1582031  -1.9667969  -2.4121094  -2.3730469  -2.6523438  -2.1582031
  -2.6835938  -2.7265625  -2.7890625  -2.9238281  -2.9628906  -1.7441406
  -2.8847656  -2.8398438  -2.5117188  -2.9472656  -2.5664062  -3.1601562
  -2.96875    -2.6816406  -2.359375   -2.2636719  -2.4160156  -2.6992188
  -2.6757812  -2.3320312  -2.5742188  -2.6660156  -2.2519531  -2.2382812
  -2.5273438  -2.0195312  -2.203125   -2.2558594  -2.5429688  -2.6054688
  -1.6191406 

In [ ]:
print("trainer.train successfully completed")


trainer.train successfully completed


### Like the above forward pass for debugging or verification purposes on dataset_dict['train'][0]['text']: recalculate inputs, outputs, logits, probs after the training.

In [ ]:
inputs = tokenizer(
    example_text,
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

print(f"inputs:\n{type(inputs)}\nlen={len(inputs)}\n{inputs.keys()}\n{inputs}")


inputs:
<class 'transformers.tokenization_utils_base.BatchEncoding'>
len=2
KeysView({'input_ids': tensor([[   0, 3721,  448,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])})
{'input_ids': tensor([[   0, 3721,  448,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


#### PyTorch tensors are created on the CPU by default.
Therefore ... device = cuda and inputs.to(device).

Not necessary in the previous check before training.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

inputs.to(device)

print(f"device: {device}")

device: cuda


In [ ]:
outputs = model(
    input_ids      = inputs.input_ids.to(device),
    attention_mask = inputs.attention_mask.to(device)
)

print(f"outputs:\n{type(outputs)}\nlen={len(outputs)}\nkeys={outputs.keys()}\n{outputs}")


outputs:
<class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'>
len=1
keys=odict_keys(['logits'])
LongformerSequenceClassifierOutput(loss={'logits': tensor([[-0.4663, -1.7656, -1.4219, -1.8115, -0.7466, -1.5283, -2.0801, -0.1462,
         -1.1123, -1.5078, -1.6025, -0.6968, -1.7266, -1.7695, -2.1934, -1.4531,
         -1.3857, -1.7471, -1.7236, -1.9707, -2.1348, -2.0195, -1.1143, -2.0859,
         -1.7119, -1.1729, -1.5420, -2.3066, -1.9551, -1.6494, -2.1289, -1.7002,
         -1.9512, -1.9131, -2.1758, -2.1699, -2.1328, -1.0010, -1.9502, -2.0156,
         -2.0059, -1.5635, -2.4590, -2.4180, -2.2988, -2.6035, -2.5879, -0.6855,
         -2.4629, -2.2109, -2.2441, -2.3789, -2.2852, -2.1973, -2.2832, -2.1602,
         -1.8926, -1.6914, -1.8408, -2.2598, -1.9014, -2.3438, -1.9941, -1.8926,
         -1.6797, -2.0625, -2.4121, -1.8818, -1.7803, -1.6279, -2.0410, -2.1152,
         -0.9316, -2.1562, -1.6553, -1.8828, -1.7480, -1.3457, -0.2966, -2.0527,


In [ ]:
# Unnecessary cell, see print of previous cell
logits = outputs.logits

print(f"logits:\n{type(logits)}\nshape={logits.shape}\n{logits}")


logits:
<class 'torch.Tensor'>
shape=torch.Size([1, 147])
tensor([[-0.4663, -1.7656, -1.4219, -1.8115, -0.7466, -1.5283, -2.0801, -0.1462,
         -1.1123, -1.5078, -1.6025, -0.6968, -1.7266, -1.7695, -2.1934, -1.4531,
         -1.3857, -1.7471, -1.7236, -1.9707, -2.1348, -2.0195, -1.1143, -2.0859,
         -1.7119, -1.1729, -1.5420, -2.3066, -1.9551, -1.6494, -2.1289, -1.7002,
         -1.9512, -1.9131, -2.1758, -2.1699, -2.1328, -1.0010, -1.9502, -2.0156,
         -2.0059, -1.5635, -2.4590, -2.4180, -2.2988, -2.6035, -2.5879, -0.6855,
         -2.4629, -2.2109, -2.2441, -2.3789, -2.2852, -2.1973, -2.2832, -2.1602,
         -1.8926, -1.6914, -1.8408, -2.2598, -1.9014, -2.3438, -1.9941, -1.8926,
         -1.6797, -2.0625, -2.4121, -1.8818, -1.7803, -1.6279, -2.0410, -2.1152,
         -0.9316, -2.1562, -1.6553, -1.8828, -1.7480, -1.3457, -0.2966, -2.0527,
         -1.9150, -1.8848, -1.5576, -1.9434, -0.4761, -2.0977, -1.7422, -1.7471,
         -1.3213, -2.0078, -1.3701, -1.7256, -1.754

In [ ]:
sigmoid = torch.nn.Sigmoid()
probs   = sigmoid(logits)

print(f"probs:\n{type(probs)}\{probs.shape}\n{probs}")

#raise Exception("LLLL")


probs:
<class 'torch.Tensor'>\torch.Size([1, 147])
tensor([[0.3855, 0.1461, 0.1944, 0.1405, 0.3216, 0.1782, 0.1110, 0.4635, 0.2474,
         0.1813, 0.1676, 0.3325, 0.1510, 0.1456, 0.1003, 0.1895, 0.2001, 0.1484,
         0.1514, 0.1223, 0.1058, 0.1172, 0.2471, 0.1105, 0.1529, 0.2363, 0.1762,
         0.0906, 0.1240, 0.1612, 0.1063, 0.1544, 0.1244, 0.1286, 0.1019, 0.1025,
         0.1059, 0.2687, 0.1245, 0.1176, 0.1186, 0.1731, 0.0788, 0.0818, 0.0912,
         0.0689, 0.0699, 0.3350, 0.0785, 0.0988, 0.0959, 0.0848, 0.0924, 0.1000,
         0.0925, 0.1034, 0.1310, 0.1556, 0.1370, 0.0945, 0.1300, 0.0876, 0.1198,
         0.1310, 0.1571, 0.1128, 0.0823, 0.1322, 0.1443, 0.1641, 0.1150, 0.1076,
         0.2826, 0.1037, 0.1604, 0.1321, 0.1483, 0.2066, 0.4264, 0.1138, 0.1284,
         0.1318, 0.1740, 0.1253, 0.3832, 0.1093, 0.1490, 0.1484, 0.2106, 0.1184,
         0.2026, 0.1512, 0.1474, 0.1180, 0.1768, 0.0962, 0.2032, 0.1212, 0.1283,
         0.1247, 0.1201, 0.1405, 0.0991, 0.1099, 0.0728, 0

#### trainer.train: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:
    name               = "trainer_train"
    trainer_train_path = f"{name}.json"

    with open(trainer_train_path, "w") as f:
        json.dump(trainer_train, f)

    print(f"{name} results successfully saved locally to {trainer_train_path}")

    upload_file(
        path_or_fileobj = trainer_train_path,
        path_in_repo    = trainer_train_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    print(f"{name} results successfully uploaded to HF Hub as {trainer_train_path}")


trainer_train results successfully saved locally to trainer_train.json
trainer_train results successfully uploaded to HF Hub as trainer_train.json


#### trainer.train: check that the uploaded file can be downloaded
File locally downloaded to:
/root/.cache/huggingface/hub/datasets-claudelepere-skill_classification/snapshots/full_commit_hash/trainer_train_results.json

In [ ]:
if ACCESS_TO_HF_HUB:
  file_path = hf_hub_download(repo_type="model", repo_id=repo_id, filename=trainer_train_path)

  print(f"file_path: {file_path}")


trainer_train.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

file_path: /root/.cache/huggingface/hub/models--claudelepere--jobs_EN_791112_50000_all_2025-07-04/snapshots/9620d87ca2a33be6dc8084880cbbf9ebcf67aa62/trainer_train.json


### Upload tokenizer and model to HF Hub


In [ ]:
if ACCESS_TO_HF_HUB:

    # Upload the tokenizer
    commit_message = f"tokenizer-{timestamp}"
    tokenizer.push_to_hub(repo_id, commit_message=commit_message)

    # Upload the model
    commit_message = f"model-{timestamp}"
    model.push_to_hub(repo_id, commit_message=commit_message)

    print(f"tokenizer and model successfully uploaded to HF Hub at {repo_id}")
    print("_____________________________________________________________________")

    # Check
    def check_upload(repo_id):
        print("Tokenizer")
        tokenizer = LongformerTokenizerFast.from_pretrained(repo_id)
        print("_________________________________________________________________")

        print("Model")
        model = LongformerForSequenceClassification.from_pretrained(repo_id)
        print("_________________________________________________________________")

        inputs = tokenizer("Hello, my dog is cute", return_tensors="pt", truncation=True, padding=True)
        outputs = model(**inputs)
        print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

    # To check if the upload was successful, download the tokenizer and the model
    check_upload(repo_id)


README.md:   0%|          | 0.00/717 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

tokenizer and model successfully uploaded to HF Hub at claudelepere/jobs_EN_791112_50000_all_2025-07-04
_____________________________________________________________________
Tokenizer


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

...

_________________________________________________________________
Model


config.json:   0%|          | 0.00/5.61k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Input ids are automatically padded to be a multiple of `config.attention_window`: 1024


_________________________________________________________________
outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> odict_keys(['logits'])
LongformerSequenceClassifierOutput(loss=None, logits=tensor([[ 0.2531, -2.2116, -1.8530, -2.1164, -0.8705, -1.8392, -2.5646, -2.0466,
         -2.2866, -1.7109, -1.9410, -1.6153, -2.1474, -2.3388, -2.3354, -1.7882,
         -1.5155, -2.2135, -2.2784, -2.1541, -2.4145, -2.4089, -1.7011, -2.2411,
         -2.0235, -1.5577, -1.7587, -2.6050, -2.1412, -2.5227, -2.3817, -2.3828,
         -2.2107, -2.1283, -2.8098, -2.3608, -2.5329, -2.0620, -2.7246, -2.6777,
         -2.7369, -2.4122, -3.0075, -2.9064, -3.1089, -3.0964, -3.1824, -1.9003,
         -2.9397, -3.2155, -2.8022, -3.1567, -2.9713, -3.2652, -3.1496, -2.8737,
         -2.5148, -2.5220, -2.6161, -3.0267, -2.7346, -2.7529, -2.6076, -2.7310,
         -2.4743, -2.3254, -2.7835, -2.3154, -2.2259, -2.2350, -2.6834, -2.6879,
         -1.7901, -2.805

### Evaluation step

#### Evaluation 1: trainer.evaluate
trainer.evaluate uses a fixed threshold of 0.5 to convert logits into binary labels, which is often suboptimal for imbalanced data.

In [ ]:
multi_label_metrics_counter = 0
compute_metrics_counter     = 0
compute_loss_counter        = 0

evaluation_trainer_evaluate_metrics = trainer.evaluate(
    #eval_dataset = encoded_dataset["validation"],  # by default, trainer.evaluate() evaluates the dataset passed as eval_dataset during training
    metric_key_prefix="eval"                       # prefix for the evaluation metrics
)

print(f"evaluation_trainer_evaluate_metrics: {type(evaluation_trainer_evaluate_metrics)} len={len(evaluation_trainer_evaluate_metrics)}\n{json.dumps(evaluation_trainer_evaluate_metrics, indent=4)}")

print(f"after trainer.evaluate(): multi_label_metrics_counter       : {multi_label_metrics_counter}")
print(f"after trainer.evaluate(): compute_metrics_counter           : {compute_metrics_counter}")
print(f"after trainer.evaluate(): compute_loss_counter              : {compute_loss_counter}")
print(f"after trainer.evaluate(): CustomTrainer_compute_loss_counter: {trainer.get_CustomTrainer_compute_loss_counter()}")
print(f"after trainer.evaluate(): FocalLoss.counter                 : {FocalLoss.counter}")

>>>>>>>>>>compute_metrics entered for the first time<<<<<<<<<<
p.predictions: <class 'numpy.ndarray'> len=5000
[[ 1.3535156  -1.3964844  -1.5722656  -2.0351562  -0.77001953 -2.0449219
  -2.4433594  -2.1835938  -2.2128906  -1.8867188  -2.1445312  -1.8974609
  -2.0371094  -2.2460938  -2.2148438  -1.5615234  -1.7792969  -2.1347656
  -2.2285156  -2.0976562  -2.3222656  -2.3652344  -1.8818359  -2.2714844
  -2.0527344  -1.5185547  -1.7392578  -2.5136719  -2.0117188  -2.3867188
  -2.2480469  -2.3828125  -2.2890625  -2.2421875  -2.6054688  -1.6767578
  -2.1582031  -1.9667969  -2.4121094  -2.3730469  -2.6523438  -2.1582031
  -2.6835938  -2.7265625  -2.7890625  -2.9238281  -2.9628906  -1.7441406
  -2.8847656  -2.8398438  -2.5117188  -2.9472656  -2.5664062  -3.1601562
  -2.96875    -2.6816406  -2.359375   -2.2636719  -2.4160156  -2.6992188
  -2.6757812  -2.3320312  -2.5742188  -2.6660156  -2.2519531  -2.2382812
  -2.5273438  -2.0195312  -2.203125   -2.2558594  -2.5429688  -2.6054688
  -1.6191406 

In [ ]:
print("evaluation 1: trainer.evaluate: successfully completed")

evaluation 1: trainer.evaluate: successfully completed


#### Evaluation 1: trainer.evaluate: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:

    name                             = "evaluation_trainer_evaluate"
    evaluation_trainer_evaluate_path = f"{name}.json"

    with open(evaluation_trainer_evaluate_path, "w") as f:
        json.dump(evaluation_trainer_evaluate_metrics, f)

    print(f"{name} successfully saved locally to {evaluation_trainer_evaluate_path}")

    upload_file(
        path_or_fileobj = evaluation_trainer_evaluate_path,
        path_in_repo    = evaluation_trainer_evaluate_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    print(f"{name} successfully uploaded to HF Hub as {evaluation_trainer_evaluate_path}")


evaluation_trainer_evaluate successfully saved locally to evaluation_trainer_evaluate.json
evaluation_trainer_evaluate successfully uploaded to HF Hub as evaluation_trainer_evaluate.json


#### Evaluation 2: trainer.predict

In [ ]:
def predict_with_optimized_thresholds(
        trainer,
        dataset,
        id2label,
        threshold_tuning = False,
        thresholds       = None
    ):
    """
    Predicts using trainer.predict(), with optional threshold tuning, using NumPy arrays, and not PyTorch tensors.

    Parameters:
    - trainer                         : Hugging Face Trainer or CustomTrainer instance
    - dataset                         : Dataset to predict on
    - id2label                        : Dictionary mapping label indices >>>>>(int)<<<<< to label names(skill Id) (str)
    - threshold_tuning                : Boolean to enable threshold tuning per class (i.e. per label)
    - thresholds       (numpy.ndarray): Custom thresholds for classification

    Returns:
    if THRESHOLD_TUNING:
        - best_thresholds      (numpy.ndarray): optimized threshold per class (label)
        - best_thresholds_dict (dict)         : optimized threshold per class (label)
        - best_metrics         (dict)         : best F1, best precision, best recall per class (label)
        - best_preds           (numpy.ndarray): best predictions per class (label)
    else:
        - thresholds      (numpy.ndarray): fixed threshold per class (label)
        - thresholds_dict (dict)         : fixed threshold per class (label)
        - metrics         (dict)         : computed with provided thresholds or default to 0.5
        - preds           (numpy.ndarray): predictions with provided thresholds or default to 0.5
    """
    # Check the type of the keys
    print(f"id2label_key_types_set: len={len(id2label_key_types_set)} {id2label_key_types_set}")  # id2label_key_types_set: len=1 {<class 'int'>}

    # Predict
    predictions_output = trainer.predict(dataset)
    logits             = predictions_output.predictions
    true_labels        = predictions_output.label_ids

    # Convert logits to probabilities (with np, not with torch)
    probs = 1 / (1 + np.exp(-logits))  # Sigmoid function

    num_labels      = probs.shape[1]  # len(id2label)
    best_thresholds = best_thresholds_dict = best_metrics = best_preds = None

    if THRESHOLD_TUNING:
        threshold_candidates = np.linspace(0.05, 0.95, 19)
        best_thresholds      = np.zeros(num_labels)
        best_metrics         = {label: {'f1': 0.0, 'precision': 0.0, 'recall': 0.0} for label in id2label.values()}

        # Iterate over each label to find the best threshold
        for label_idx, label in id2label.items():
            # Predictions for the current label across all threshold candidates
            preds = probs[:, label_idx][:, None] > threshold_candidates  # Create a matrix of shape (num_samples, num_thresholds)

            # Compute precision, recall, F1 for all thresholds at once for the current label
            precision, recall, f1, _ = precision_recall_fscore_support(
                np.tile(true_labels[:, label_idx], (len(threshold_candidates), 1)).T, preds, average=None, zero_division='warn'
            )

            # Find the best threshold based on F1 for the current label
            best_idx                   = np.argmax(f1)
            best_thresholds[label_idx] = threshold_candidates[best_idx]
            best_metrics[label]        = {'f1': f1[best_idx], 'precision': precision[best_idx], 'recall': recall[best_idx]}

        best_thresholds_dict = {id2label[i]: best_thresholds[i].item() for i in range(len(best_thresholds))}

        # Generate predictions using the optimized threshold for each label
        #best_preds = np.zeros_like(true_labels, dtype=int)
        #for label_idx, label in id2label.items():
        #    best_preds[:, label_idx] = (probs[:, label_idx] > best_thresholds[label_idx]).astype(int)
        best_preds = np.array([(probs[:, i] > best_thresholds[i]).astype(int) for i in range(num_labels)]).T

        #print("==== best_thresholds, best_threshold_dict and best_metrics ====")
        #print(f"best_thresholds:      {type(best_thresholds)} shape={best_thresholds.shape}\n{best_thresholds}")                # <class 'numpy.ndarray'> shape=(6,)
        #print(f"best_thresholds_dict: {type(best_thresholds_dict)} len={len(best_thresholds_dict)}\n{best_thresholds_dict}")    # <class 'dict'> len=6
        #print(f"best_metrics:         {type(best_metrics)} len={len(best_metrics)}\n{json.dumps(best_metrics, indent=4)}")      # <class 'dict'> len=6
        #print("===============================================================")
        #print()

    # ==== If not THRESHOLD_TUNING ====

    # Apply provided thresholds or default to 0.5
    thresholds_fixed = thresholds if thresholds is not None else np.full(num_labels, 0.5)

    # Compute predictions with fixed thresholds
    preds_fixed = (probs > thresholds_fixed).astype(int)

    # Compute metrics in one step (no loop)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds_fixed, average=None, zero_division='warn')

    # Convert to dict format
    metrics_fixed         = {id2label[i]: {'f1': f1[i], 'precision': precision[i], 'recall': recall[i]} for i in range(num_labels)}
    thresholds_fixed_dict = {id2label[i]: thresholds_fixed[i].item() for i in range(num_labels)}

    #print("==== provided thresholds and metrics ====")
    #print(f"thresholds_fixed     : {type(thresholds_fixed)} shape={thresholds_fixed.shape}\n{thresholds_fixed}")              # <class 'numpy.ndarray'> shape=(6,)
    #print(f"thresholds_fixed_dict: {type(thresholds_fixed_dict)} len={len(thresholds_fixed_dict)}\n{thresholds_fixed_dict}")  # <class 'dict'> len=6
    #print(f"metrics_fixed        : {type(metrics_fixed)} len={len(metrics_fixed)}\n{json.dumps(metrics_fixed, indent=4)}")    # <class 'dict'> len=6
    #print("===============================================================")
    #print()

    thresholds      = best_thresholds      if THRESHOLD_TUNING else thresholds_fixed
    thresholds_dict = best_thresholds_dict if THRESHOLD_TUNING else thresholds_fixed_dict
    metrics         = best_metrics         if THRESHOLD_TUNING else metrics_fixed
    preds           = best_preds           if THRESHOLD_TUNING else preds_fixed

    # Compute micro average
    #   compute metrics globally by summing all TP, FP, FN across all labels
    #   good for overall performance assessment
    #   dominated by frequent labels: if most samples belong to a few labels, if favors them
    #micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
    #    true_labels, preds, average='micro', zero_division=0
    #)

    # Compute macro average
    #   each label is treated equally, regardless of how often it appears
    #   good for evaluating rare labels
    #   sensitive to rare labels: if rare labels perform poorly, macro F1 will drop
    #macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
    #    true_labels, preds, average='macro', zero_division=0
    #)

    # Compute weighted average
    #   like macro, but weights each label's F1 based on its frequency
    #   balances between micro and macro by considering both label importance and prevalence
    #   useful if class imbalance exists but you still want per-label influence
    #weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(
    #    true_labels, preds, average='weighted', zero_division=0
    #)

    #global_metrics = {
    #    'micro':    {'f1': micro_f1,    'precision': micro_precision,    'recall': micro_recall},
    #    'macro':    {'f1': macro_f1,    'precision': macro_precision,    'recall': macro_recall},
    #    'weighted': {'f1': weighted_f1, 'precision': weighted_precision, 'recall': weighted_recall}
    #}

    global_metrics = {}
    for avg in ['micro', 'macro', 'weighted', 'samples']:
        p, r, f, _ = precision_recall_fscore_support(true_labels, preds, average=avg, zero_division='warn')
        global_metrics[avg] = {'f1': f, 'precision': p, 'recall': r}


    return thresholds, thresholds_dict, metrics, global_metrics


#### Evaluation 2: calculate metrics and optimized thresholds

- First, to **calculate** the optimized thresholds, threshold_tuning = True and thresholds = None.
- After, to **use** these optimized thresholds, threshold_tuning = False and thresholds = the optimized thresholds.

In [ ]:
multi_label_metrics_counter = 0
compute_metrics_counter     = 0
compute_loss_counter        = 0

# with best_thresholds (threshold_tuning = True and thresholds = None)
optimized_thresholds, optimized_thresholds_dict, evaluation_trainer_predict_metrics, evaluation_trainer_predict_global_metrics = predict_with_optimized_thresholds(
    trainer, validation_dataset, id2label, threshold_tuning=True, thresholds=None)

print("==== with best thresholds ====")
print(f"optimized_thresholds                     : {type(optimized_thresholds)} shape={optimized_thresholds.shape} {optimized_thresholds}")
print(f"optimized_thresholds_dict                : {type(optimized_thresholds_dict)} len={len(optimized_thresholds_dict)}\n{optimized_thresholds_dict}")
print(f"evaluation_trainer_predict_metrics       : {type(evaluation_trainer_predict_metrics)} len={len(evaluation_trainer_predict_metrics)}\n{json.dumps(evaluation_trainer_predict_metrics, indent=4)}")
print(f"evaluation_trainer_predict_global_metrics: {type(evaluation_trainer_predict_global_metrics)} len={len(evaluation_trainer_predict_global_metrics)}\n{json.dumps(evaluation_trainer_predict_global_metrics, indent=4)}")

print(f"after trainer.predict(): after best thresholds: multi_label_metrics_counter       : {multi_label_metrics_counter}")
print(f"after trainer.predict(): after best thresholds: compute_metrics_counter           : {compute_metrics_counter}")
print(f"after trainer.predict(): after best thresholds: compute_loss_counter              : {compute_loss_counter}")
print(f"after trainer.predict(): after best thresholds: CustomTrainer_compute_loss_counter: {trainer.get_CustomTrainer_compute_loss_counter()}")
print(f"after trainer.predict(): after best thresholds: FocalLoss.counter                 : {FocalLoss.counter}")
print("__________________________________________________________________________________")
print()

multi_label_metrics_counter = 0
compute_metrics_counter     = 0
compute_loss_counter        = 0

# with thresholds_fixed=0.5 (threshold_tuning = False and thresholds = None)
thresholds, thresholds_dict, evaluation_trainer_predict_metrics_thr05, evaluation_trainer_predict_global_metrics_thr05 = predict_with_optimized_thresholds(
    trainer, validation_dataset, id2label, threshold_tuning=False, thresholds=None)

print("==== with default fixed thresholds = 0.5 ====")
print(f"thresholds                                     : {type(thresholds)} shape={thresholds.shape} {thresholds}")
print(f"thresholds_dict                                : {type(thresholds_dict)} len={len(thresholds_dict)}\n{thresholds_dict}")
print(f"evaluation_trainer_predict_metrics_thr05       : {type(evaluation_trainer_predict_metrics_thr05)} len={len(evaluation_trainer_predict_metrics_thr05)}\n{json.dumps(evaluation_trainer_predict_metrics_thr05, indent=4)}")
print(f"evaluation_trainer_predict_global_metrics_thr05: {type(evaluation_trainer_predict_global_metrics_thr05)} len={len(evaluation_trainer_predict_global_metrics_thr05)}\n{json.dumps(evaluation_trainer_predict_global_metrics_thr05, indent=4)}")

print(f"after trainer.predict(): after 0.5 fixed threshold: multi_label_metrics_counter       : {multi_label_metrics_counter}")
print(f"after trainer.predict(): after 0.5 fixed threshold: compute_metrics_counter           : {compute_metrics_counter}")
print(f"after trainer.predict(): after 0.5 fixed threshold: compute_loss_counter              : {compute_loss_counter}")
print(f"after trainer.predict(): after 0.5 fixed threshold: CustomTrainer_compute_loss_counter: {trainer.get_CustomTrainer_compute_loss_counter()}")
print(f"after trainer.predict(): after 0.5 fixed threshold: FocalLoss.counter                 : {FocalLoss.counter}")

print("__________________________________________________________________________________")

#raise Exception("ZZZZ")

id2label_key_types_set: len=1 {<class 'int'>}


>>>>>>>>>>compute_metrics entered for the first time<<<<<<<<<<
p.predictions: <class 'numpy.ndarray'> len=5000
[[ 1.3535156  -1.3964844  -1.5722656  -2.0351562  -0.77001953 -2.0449219
  -2.4433594  -2.1835938  -2.2128906  -1.8867188  -2.1445312  -1.8974609
  -2.0371094  -2.2460938  -2.2148438  -1.5615234  -1.7792969  -2.1347656
  -2.2285156  -2.0976562  -2.3222656  -2.3652344  -1.8818359  -2.2714844
  -2.0527344  -1.5185547  -1.7392578  -2.5136719  -2.0117188  -2.3867188
  -2.2480469  -2.3828125  -2.2890625  -2.2421875  -2.6054688  -1.6767578
  -2.1582031  -1.9667969  -2.4121094  -2.3730469  -2.6523438  -2.1582031
  -2.6835938  -2.7265625  -2.7890625  -2.9238281  -2.9628906  -1.7441406
  -2.8847656  -2.8398438  -2.5117188  -2.9472656  -2.5664062  -3.1601562
  -2.96875    -2.6816406  -2.359375   -2.2636719  -2.4160156  -2.6992188
  -2.6757812  -2.3320312  -2.5742188  -2.6660156  -2.2519531  -2.2382812
  -2.5273438  -2.0195312  -2.203125   -2.2558594  -2.5429688  -2.6054688
  -1.6191406 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

==== with best thresholds ====
optimized_thresholds                     : <class 'numpy.ndarray'> shape=(147,) [0.5  0.2  0.4  0.35 0.4  0.35 0.1  0.35 0.25 0.4  0.4  0.4  0.4  0.4
 0.15 0.3  0.45 0.25 0.2  0.3  0.25 0.2  0.4  0.2  0.3  0.25 0.3  0.1
 0.2  0.2  0.2  0.2  0.3  0.4  0.15 0.15 0.15 0.35 0.15 0.1  0.2  0.3
 0.05 0.1  0.05 0.05 0.05 0.3  0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.1  0.15 0.2  0.15 0.2  0.1  0.4  0.25 0.15 0.3  0.1  0.3  0.35 0.3
 0.2  0.1  0.35 0.1  0.2  0.1  0.15 0.4  0.3  0.05 0.15 0.4  0.2  0.2
 0.25 0.1  0.3  0.15 0.4  0.25 0.35 0.3  0.35 0.05 0.2  0.15 0.35 0.2
 0.25 0.15 0.35 0.3  0.3  0.15 0.1  0.35 0.2  0.2  0.05 0.05 0.4  0.25
 0.05 0.1  0.2  0.25 0.1  0.25 0.25 0.3  0.3  0.15 0.25 0.25 0.3  0.25
 0.15 0.45 0.25 0.25 0.35 0.1  0.1  0.15 0.15 0.25 0.3  0.3  0.3  0.35
 0.05 0.05 0.5  0.2  0.2  0.05 0.3 ]
optimized_thresholds_dict                : <class 'dict'> len=147
{'142': 0.49999999999999994, '143': 0.2, '146': 0.39999999999999997, '147': 0.35, 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


>>>>>>>>>>compute_metrics entered for the first time<<<<<<<<<<
p.predictions: <class 'numpy.ndarray'> len=5000
[[ 1.3535156  -1.3964844  -1.5722656  -2.0351562  -0.77001953 -2.0449219
  -2.4433594  -2.1835938  -2.2128906  -1.8867188  -2.1445312  -1.8974609
  -2.0371094  -2.2460938  -2.2148438  -1.5615234  -1.7792969  -2.1347656
  -2.2285156  -2.0976562  -2.3222656  -2.3652344  -1.8818359  -2.2714844
  -2.0527344  -1.5185547  -1.7392578  -2.5136719  -2.0117188  -2.3867188
  -2.2480469  -2.3828125  -2.2890625  -2.2421875  -2.6054688  -1.6767578
  -2.1582031  -1.9667969  -2.4121094  -2.3730469  -2.6523438  -2.1582031
  -2.6835938  -2.7265625  -2.7890625  -2.9238281  -2.9628906  -1.7441406
  -2.8847656  -2.8398438  -2.5117188  -2.9472656  -2.5664062  -3.1601562
  -2.96875    -2.6816406  -2.359375   -2.2636719  -2.4160156  -2.6992188
  -2.6757812  -2.3320312  -2.5742188  -2.6660156  -2.2519531  -2.2382812
  -2.5273438  -2.0195312  -2.203125   -2.2558594  -2.5429688  -2.6054688
  -1.6191406 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

==== with default fixed thresholds = 0.5 ====
thresholds                                     : <class 'numpy.ndarray'> shape=(147,) [0.5  0.2  0.4  0.35 0.4  0.35 0.1  0.35 0.25 0.4  0.4  0.4  0.4  0.4
 0.15 0.3  0.45 0.25 0.2  0.3  0.25 0.2  0.4  0.2  0.3  0.25 0.3  0.1
 0.2  0.2  0.2  0.2  0.3  0.4  0.15 0.15 0.15 0.35 0.15 0.1  0.2  0.3
 0.05 0.1  0.05 0.05 0.05 0.3  0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.1  0.15 0.2  0.15 0.2  0.1  0.4  0.25 0.15 0.3  0.1  0.3  0.35 0.3
 0.2  0.1  0.35 0.1  0.2  0.1  0.15 0.4  0.3  0.05 0.15 0.4  0.2  0.2
 0.25 0.1  0.3  0.15 0.4  0.25 0.35 0.3  0.35 0.05 0.2  0.15 0.35 0.2
 0.25 0.15 0.35 0.3  0.3  0.15 0.1  0.35 0.2  0.2  0.05 0.05 0.4  0.25
 0.05 0.1  0.2  0.25 0.1  0.25 0.25 0.3  0.3  0.15 0.25 0.25 0.3  0.25
 0.15 0.45 0.25 0.25 0.35 0.1  0.1  0.15 0.15 0.25 0.3  0.3  0.3  0.35
 0.05 0.05 0.5  0.2  0.2  0.05 0.3 ]
thresholds_dict                                : <class 'dict'> len=147
{'142': 0.49999999999999994, '143': 0.2, '146': 0.39999

In [ ]:
print("evaluation 2: trainer.predict: successfully completed")


evaluation 2: trainer.predict: successfully completed


#### Evaluation 2: trainer.predict: save locally and upload to HF Hub


In [ ]:
if ACCESS_TO_HF_HUB:
    name                            = "evaluation_trainer_predict"
    evaluation_trainer_predict_path = f"{name}.json"

    with open(evaluation_trainer_predict_path, "w") as f:
        json.dump(evaluation_trainer_predict_metrics, f)

    print(f"{name} successfully saved locally to {evaluation_trainer_predict_path}")

    upload_file(
        path_or_fileobj = evaluation_trainer_predict_path,
        path_in_repo    = evaluation_trainer_predict_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    print(f"{name} successfully uploaded to HF Hub as {evaluation_trainer_predict_path}")

    """
    name_thr05                            = "evaluation_trainer_predict_thr05"
    evaluation_trainer_predict_path_thr05 = f"{name_thr05}.json"

    with open(evaluation_trainer_predict_path_thr05, "w") as f:
        json.dump(evaluation_trainer_predict_metrics_thr05, f)

    print(f"{name_thr05} successfully saved locally to {evaluation_trainer_predict_path_thr05}")

    upload_file(
        path_or_fileobj = evaluation_trainer_predict_path_thr05,
        path_in_repo    = evaluation_trainer_predict_path_thr05,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name_thr05}-{timestamp}"
    )

    print(f"{name_thr05} successfully uploaded to HF Hub as {evaluation_trainer_predict_path_thr05}")
    """


evaluation_trainer_predict successfully saved locally to evaluation_trainer_predict.json
evaluation_trainer_predict successfully uploaded to HF Hub as evaluation_trainer_predict.json


#### Evaluation 2: optimized thresholds: save locally (as a dict) and upload to HF Hub (as a JSON file in repo 'dataset')
optimized_thresholds: <class 'numpy.ndarray'> shape=(6,) but JSON doesn't support NumPy types

In [ ]:
if THRESHOLD_TUNING and ACCESS_TO_HF_HUB:
    name                      = "optimized_thresholds"
    optimized_thresholds_path = f"{name}.json"

    with open(optimized_thresholds_path, "w") as f:
        json.dump(optimized_thresholds_dict, f, indent=4)

    print(f"{optimized_thresholds_dict} successfully saved locally to {optimized_thresholds_path}")

    upload_file(
        path_or_fileobj = optimized_thresholds_path,
        path_in_repo    = f"optimized_thresholds/{optimized_thresholds_path}",
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    #files.download(optimized_thresholds_path)

    print(f"{name} successfully uploaded to HF Hub as {optimized_thresholds_path}")


{'142': 0.49999999999999994, '143': 0.2, '146': 0.39999999999999997, '147': 0.35, '149': 0.39999999999999997, '150': 0.35, '152': 0.1, '154': 0.35, '155': 0.25, '156': 0.39999999999999997, '158': 0.39999999999999997, '160': 0.39999999999999997, '162': 0.39999999999999997, '165': 0.39999999999999997, '167': 0.15, '169': 0.3, '170': 0.44999999999999996, '171': 0.25, '173': 0.2, '174': 0.3, '175': 0.25, '176': 0.2, '356': 0.39999999999999997, '360': 0.2, '361': 0.3, '362': 0.25, '364': 0.3, '370': 0.1, '371': 0.2, '373': 0.2, '375': 0.2, '376': 0.2, '394': 0.3, '408': 0.39999999999999997, '667': 0.15, '686': 0.15, '687': 0.15, '689': 0.35, '756': 0.15, '758': 0.1, '760': 0.2, '761': 0.3, '900': 0.05, '901': 0.1, '902': 0.05, '903': 0.05, '905': 0.05, '906': 0.3, '907': 0.05, '908': 0.05, '909': 0.05, '911': 0.05, '912': 0.05, '913': 0.05, '914': 0.05, '916': 0.05, '263': 0.1, '265': 0.15, '266': 0.2, '267': 0.15, '268': 0.2, '270': 0.1, '271': 0.39999999999999997, '272': 0.25, '273': 0.15

#### Evaluation 3: model.eval

In [ ]:
def compute_metrics_with_threshold(model, dataset, optimized_thresholds, id2label, batch_size=8):
    """
    Compute metrics during evaluation or test, by applying tuned thresholds

    Parameters:
    - model                                               : Hugging Face model
    - dataset                                             : Dataset to predict on
    - optimized_thresholds (list or NumPy array of floats): Optimized thresholds for each label
    - id2label                                            : Dictionary mapping label indices (int) to label names (string)
    - batch_size                                          : Batch size for prediction. Defaults to 8
    Returns:
    - metrics (dict)

    Compute metrics during evaluation or test, by applying optimized thresholds
    """
    dataloader = DataLoader(dataset, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)    # Move model to GPU/CPU
    model.eval()        # Set model to evaluation mode

    all_logits, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            labels  = batch.pop('labels')                           # Keep labels on CPU
            inputs  = {k: v.to(device) for k, v in batch.items()}   # Move inputs to device
            outputs = model(**inputs)

            all_logits.append(outputs.logits.cpu())     # Keep logits as tensors, move to CPU
            all_labels.append(labels)                   # Labels remain on CPU

    # Stack tensors
    logits = torch.cat(all_logits, dim=0).to(device)   # shape = (num_samples, num_labels), move to device
    labels = torch.cat(all_labels, dim=0).to(device)   # shape = (num_samples, num_labels), move to device

    # Convert logits to probabilities
    probs = torch.sigmoid(logits)           # shape = (num_samples, num_labels)

    # Apply per-class tuned thresholds (element-wise comparison)
    thresholds = torch.tensor(optimized_thresholds, dtype=torch.float32, device=device)  # Convert tuned_thresholds to tensor
    preds      = (probs > thresholds).int()                                              # Convert to binary predictions (1 or 0)

    # Compute TP, FP, FN, TN
    TP = ((preds == 1) & (labels == 1)).sum(dim=0).float()
    TN = ((preds == 0) & (labels == 0)).sum(dim=0).float()
    FP = ((preds == 1) & (labels == 0)).sum(dim=0).float()
    FN = ((preds == 0) & (labels == 1)).sum(dim=0).float()
    print(f"FN: {type(FN)} {FN}")
    print(f"TP: {type(TP)} {TP}")
    print(f"TN: {type(TN)} {TN}")
    print(f"FP: {type(FP)} {FP}")

    # Compute per-class metrics
    precision_per_class = TP / (TP + FP + 1e-8)
    recall_per_class    = TP / (TP + FN + 1e-8)
    f1_per_class        = 2 * (precision_per_class * recall_per_class) / (precision_per_class + recall_per_class + 1e-8)

    # Compute averaged metrics
    precision = precision_per_class.mean()
    recall    = recall_per_class.mean()
    f1        = f1_per_class.mean()

    # Compute accuracy
    accuracy = (preds == labels).float().mean()

    # Convert to NumPy for ROC-AUC and PR-AUC
    labels_np = labels.cpu().numpy()  # Move to CPU before converting
    probs_np  = probs.cpu().numpy()   # Move to CPU before converting

    # Compute ROC-AUC and PR-AUC
    roc_auc = torch.tensor(roc_auc_score(labels_np, probs_np, average=evaluation_average, multi_class='ovr'))
    pr_auc  = torch.tensor(average_precision_score(labels_np, probs_np, average=evaluation_average))

    # Convert predictions to Numpy for classification_report
    preds_np = preds.cpu().numpy()  # Move to CPU before converting

    # Generate classification report
    class_names  = [id2label[i] for i in range(len(id2label))]
    class_report = classification_report(labels_np, preds_np, target_names=class_names, zero_division=0)

    #print(f"\nClassification Report:\n{class_report}")

    # Store metrics
    metrics = {
        'accurary'             : accuracy.item(),
        'precision'            : precision.item(),
        'recall'               : recall.item(),
        'f1'                   : f1.item(),
        'roc_auc'              : roc_auc.item(),
        'pr_auc'               : pr_auc.item(),
        'per_class_precision'  : {id2label[i]: precision_per_class[i].item() for i in range(len(id2label))},
        'per_class_recall'     : {id2label[i]: recall_per_class[i].item() for i in range(len(id2label))},
        'per_class_f1'         : {id2label[i]: f1_per_class[i].item() for i in range(len(id2label))},
        'classification_report': class_report,
    }

    return metrics


In [ ]:
multi_label_metrics_counter = 0
compute_metrics_counter     = 0
compute_loss_counter        = 0

evaluation_model_eval_metrics = compute_metrics_with_threshold(model, validation_dataset, optimized_thresholds, id2label, batch_size=16)

except_report = {k: v for k, v in evaluation_model_eval_metrics.items() if k!='classification_report'}
report        = evaluation_model_eval_metrics['classification_report']
print(f"evaluation_model_eval_metrics: {type(except_report)} len={len(except_report)}\n{json.dumps(except_report, indent=4)}")
print(f"evaluation_model_eval_metrics['classification_report']: {type(report)} len={len(report)}\n{report}")

print(f"after model.eval(): multi_label_metrics_counter       : {multi_label_metrics_counter}")
print(f"after model.eval(): compute_metrics_counter           : {compute_metrics_counter}")
print(f"after model.eval(): compute_loss_counter              : {compute_loss_counter}")
print(f"after model.eval(): CustomTrainer_compute_loss_counter: {trainer.get_CustomTrainer_compute_loss_counter()}")
print(f"after model.eval(): FocalLoss.counter                 : {FocalLoss.counter}")


FN: <class 'torch.Tensor'> tensor([253.,  40., 141.,  93., 350.,  50.,   5.,  43.,  26., 110., 119., 181.,
         58.,  93.,  24., 139., 172.,  37.,  43.,  55.,  35.,  36., 145.,  12.,
         43., 101., 118.,   4.,  30.,  49.,  47.,  40.,  65.,  75.,  15.,  28.,
         20.,  69.,  12.,   7.,  22.,  28.,   0.,   0.,   0.,   0.,   0.,  74.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  14.,  17.,  29.,  11.,
         32.,   2.,  54.,  24.,  22.,  48.,   4.,  57.,  54.,  81.,  20.,   6.,
        109.,   9.,  49.,   6.,  40., 134.,  98.,   0.,  11., 118.,  73.,  27.,
        102.,   5.,  73.,  26., 123.,  39., 110.,  51.,  40.,   0.,  19.,   7.,
        121.,  31.,  43.,  15.,  57.,  56.,  36.,  11.,   3., 115.,  45.,  34.,
          0.,   0., 195.,  66.,   0.,  13.,  11.,  30.,   4.,  17.,  50.,  42.,
         36.,  11.,  46.,  29.,  24.,  13.,  31., 108.,  19.,  35.,  21.,   3.,
          2.,   8.,  11.,  11.,  24., 189., 226., 320.,   0.,   0.,  73.,  84.,
        131.,

In [ ]:
print("evaluation 3: model.eval: successfully completed")

evaluation 3: model.eval: successfully completed


#### Evaluation 3: model.eval: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:

    name                       = "evaluation_model_eval"
    evaluation_model_eval_path = f"{name}.json"

    with open(evaluation_model_eval_path, "w") as f:
        json.dump(evaluation_model_eval_metrics, f)

    print(f"{name} successfully saved locally to {evaluation_model_eval_path}")

    upload_file(
        path_or_fileobj = evaluation_model_eval_path,
        path_in_repo    = evaluation_model_eval_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    print(f"{name} successfully uploaded to HF Hub as {evaluation_model_eval_path}")


evaluation_model_eval successfully saved locally to evaluation_model_eval.json
evaluation_model_eval successfully uploaded to HF Hub as evaluation_model_eval.json


### Test step

#### Test 1: trainer.evaluate

In [ ]:
test_trainer_evaluate_metrics = trainer.evaluate(
    eval_dataset = encoded_dataset['test'],
    metric_key_prefix='test'
)

print(f"test_trainer_evaluate_metrics: {type(test_trainer_evaluate_metrics)} len={len(test_trainer_evaluate_metrics)}\n{json.dumps(test_trainer_evaluate_metrics, indent=4)}")


>>>>>>>>>>compute_metrics entered for the first time<<<<<<<<<<
p.predictions: <class 'numpy.ndarray'> len=5000
[[-0.30981445 -1.9248047  -1.4199219  -2.0878906  -0.03022766 -1.9521484
  -2.4160156  -1.1601562  -1.3984375  -1.8457031  -2.0273438  -1.6787109
  -2.0390625  -2.2597656  -2.2265625  -1.3271484  -1.4775391  -1.9316406
  -1.7939453  -1.8740234  -2.1835938  -2.2089844  -0.21350098 -2.2226562
  -1.6855469  -1.4316406  -1.0244141  -2.5917969  -1.9980469  -2.0039062
  -2.2851562  -1.8945312  -2.1445312  -2.0957031  -2.390625   -1.9501953
  -2.234375   -0.30615234 -2.2226562  -2.296875   -2.0703125  -1.9316406
  -2.6933594  -2.6210938  -2.7070312  -2.8359375  -2.6816406  -1.0078125
  -2.7402344  -2.7480469  -2.3574219  -2.7285156  -2.4589844  -2.78125
  -2.9042969  -2.609375   -1.9941406  -2.0078125  -2.3789062  -2.5859375
  -2.2753906  -2.5195312  -2.4492188  -2.328125   -1.9238281  -2.5390625
  -2.6015625  -2.5195312  -2.3398438  -2.2089844  -2.6132812  -2.4433594
  -1.6708984  -

In [ ]:
print("test_trainer.evaluate successfully completed")

test_trainer.evaluate successfully completed


#### Test 1: trainer.evaluate: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:

    name                       = "test_trainer_evaluate"
    test_trainer_evaluate_path = f"{name}.json"

    with open(test_trainer_evaluate_path, "w") as f:
        json.dump(test_trainer_evaluate_metrics, f)

    print(f"{name} results successfully saved locally to {test_trainer_evaluate_path}")

    upload_file(
        path_or_fileobj = test_trainer_evaluate_path,
        path_in_repo    = test_trainer_evaluate_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    print(f"{name} results successfully uploaded to HF Hub as {test_trainer_evaluate_path}")


test_trainer_evaluate results successfully saved locally to test_trainer_evaluate.json
test_trainer_evaluate results successfully uploaded to HF Hub as test_trainer_evaluate.json


#### Test 2: trainer.predict

In [ ]:
optimized_thresholds, optimized_thresholds_dict, test_trainer_predict_metrics, test_trainer_predict_global_metrics = predict_with_optimized_thresholds(
    trainer, test_dataset, id2label, threshold_tuning=False, thresholds=optimized_thresholds)

print(f"optimized_thresholds: {type(optimized_thresholds)} shape={optimized_thresholds.shape} {optimized_thresholds}")
print(f"optimized_thresholds_dict: {type(optimized_thresholds_dict)} len={len(optimized_thresholds_dict)}\n{optimized_thresholds_dict}")
print(f"test_trainer_predict_metrics: {type(test_trainer_predict_metrics)} len={len(test_trainer_predict_metrics)}\n{json.dumps(test_trainer_predict_metrics, indent=4)}")


id2label_key_types_set: len=1 {<class 'int'>}


p.predictions: <class 'numpy.ndarray'> len=5000
[[-0.30981445 -1.9248047  -1.4199219  -2.0878906  -0.03022766 -1.9521484
  -2.4160156  -1.1601562  -1.3984375  -1.8457031  -2.0273438  -1.6787109
  -2.0390625  -2.2597656  -2.2265625  -1.3271484  -1.4775391  -1.9316406
  -1.7939453  -1.8740234  -2.1835938  -2.2089844  -0.21350098 -2.2226562
  -1.6855469  -1.4316406  -1.0244141  -2.5917969  -1.9980469  -2.0039062
  -2.2851562  -1.8945312  -2.1445312  -2.0957031  -2.390625   -1.9501953
  -2.234375   -0.30615234 -2.2226562  -2.296875   -2.0703125  -1.9316406
  -2.6933594  -2.6210938  -2.7070312  -2.8359375  -2.6816406  -1.0078125
  -2.7402344  -2.7480469  -2.3574219  -2.7285156  -2.4589844  -2.78125
  -2.9042969  -2.609375   -1.9941406  -2.0078125  -2.3789062  -2.5859375
  -2.2753906  -2.5195312  -2.4492188  -2.328125   -1.9238281  -2.5390625
  -2.6015625  -2.5195312  -2.3398438  -2.2089844  -2.6132812  -2.4433594
  -1.6708984  -2.4960938  -2.1113281  -2.3378906  -2.1152344  -1.9785156
  -1.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

optimized_thresholds: <class 'numpy.ndarray'> shape=(147,) [0.5  0.2  0.4  0.35 0.4  0.35 0.15 0.4  0.25 0.4  0.35 0.45 0.4  0.45
 0.2  0.3  0.45 0.25 0.2  0.3  0.3  0.2  0.4  0.2  0.25 0.25 0.3  0.1
 0.2  0.2  0.2  0.2  0.3  0.35 0.1  0.2  0.15 0.4  0.15 0.1  0.2  0.3
 0.05 0.05 0.05 0.05 0.05 0.3  0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.1  0.15 0.2  0.15 0.2  0.1  0.4  0.25 0.15 0.35 0.1  0.3  0.35 0.3
 0.15 0.15 0.4  0.15 0.2  0.15 0.15 0.4  0.3  0.05 0.15 0.4  0.2  0.2
 0.3  0.05 0.3  0.2  0.45 0.2  0.4  0.3  0.35 0.05 0.15 0.15 0.3  0.2
 0.25 0.15 0.35 0.3  0.3  0.1  0.1  0.3  0.2  0.2  0.1  0.1  0.4  0.25
 0.05 0.15 0.2  0.2  0.1  0.25 0.3  0.35 0.25 0.15 0.25 0.25 0.3  0.3
 0.15 0.4  0.25 0.25 0.25 0.1  0.15 0.15 0.15 0.25 0.3  0.25 0.3  0.35
 0.45 0.05 0.45 0.2  0.2  0.4  0.3 ]
optimized_thresholds_dict: <class 'dict'> len=147
{'142': 0.49999999999999994, '143': 0.2, '146': 0.39999999999999997, '147': 0.35, '149': 0.39999999999999997, '150': 0.35, '152': 0.15, '154': 0.39999

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print("test_trainer.predict successfully completed")


test_trainer.predict successfully completed


#### Test 2: trainer.predict: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:

    name                      = "test_trainer_predict"
    test_trainer_predict_path = f"{name}.json"

    with open(test_trainer_predict_path, "w") as f:
        json.dump(test_trainer_predict_metrics, f)

    print(f"{name} results successfully saved locally to {test_trainer_predict_path}")

    upload_file(
        path_or_fileobj = test_trainer_predict_path,
        path_in_repo    = test_trainer_predict_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    print(f"{name} results successfully uploaded to HF Hub as {test_trainer_predict_path}")


test_trainer_predict results successfully saved locally to test_trainer_predict.json
test_trainer_predict results successfully uploaded to HF Hub as test_trainer_predict.json


#### Test 3: model.eval

In [ ]:
test_model_eval_metrics = compute_metrics_with_threshold(model, test_dataset, optimized_thresholds, id2label, batch_size=16)

except_report = {k: v for k, v in test_model_eval_metrics.items() if k!='classification_report'}
report        = test_model_eval_metrics['classification_report']
print(f"test_model_eval_metrics: {type(except_report)} len={len(except_report)}\n{json.dumps(except_report, indent=4)}")
print(f"test_model_eval_metrics['classification_report']: {type(report)} len={len(report)}\n{report}")


FN: <class 'torch.Tensor'> tensor([235.,  26., 141., 101., 311.,  48.,  16.,  60.,  41., 101.,  98., 227.,
         50.,  88.,  46., 116., 164.,  30.,  37.,  73.,  66.,  33., 148.,  11.,
         45., 116., 118.,   7.,  31.,  49.,  35.,  37.,  54.,  55.,   3.,  25.,
         16.,  71.,  10.,   8.,  22.,  34.,   0.,   0.,   0.,   0.,   0.,  78.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,  19.,  17.,   8.,
         19.,   1.,  50.,  28.,  22.,  47.,   2.,  53.,  51.,  74.,   6.,  16.,
        170.,  17.,  53.,  13.,  34., 155., 119.,   0.,  11., 108.,  72.,  19.,
        164.,   0.,  57.,  29., 142.,  11., 163.,  56.,  43.,   0.,  13.,   8.,
        133.,  32.,  32.,  22.,  49.,  65.,  42.,   3.,   4.,  81.,  41.,  34.,
          4.,   3., 187.,  61.,   0.,  21.,  10.,  23.,   2.,  21.,  58.,  59.,
         30.,  11.,  44.,  34.,  25.,  22.,  32., 107.,  22.,  26.,  12.,   3.,
         23.,  19.,  18.,   8.,  15., 141., 236., 348.,   1.,   0.,   3.,  71.,
        123.,

#### Test 3: model.eval: save locally and upload to HF Hub

In [ ]:
if ACCESS_TO_HF_HUB:

    name                 = "test_model_eval"
    test_model_eval_path = f"{name}.json"

    with open(test_model_eval_path, "w") as f:
        json.dump(test_model_eval_metrics, f)

    print(f"{name} successfully saved locally to {test_model_eval_path}")

    upload_file(
        path_or_fileobj = test_model_eval_path,
        path_in_repo    = test_model_eval_path,
        repo_id         = repo_id,
        repo_type       = 'model',
        commit_message  = f"{name}-{timestamp}"
    )

    print(f"{name} successfully uploaded to HF Hub as {test_model_eval_path}")


test_model_eval successfully saved locally to test_model_eval.json
test_model_eval successfully uploaded to HF Hub as test_model_eval.json


### Upload to HF Hub: blocking and no--blocking
- See Evernote: Hugging Face Hub: are push_to_hub and upload_file upload_folder blocking?
- push_to_hub (for models/datasets): blocking by default
- upload_file and upload_folder: blocking by default
- => runtime.unassign() as last cell is safe

In [ ]:
#raise Exception("It's the end, I stop here")
print("It's the end.")
#sys.exit(0)  # doesn't disconnect the runtime, doesn't stop consuming compute unit (Colab auto-disconnets after ~90 minutes)
#time.sleep(300)  # Wait 5 minutes before shutdown
#os.kill(os.getpid(), 9)
runtime.unassign()

It's the end.


==========================================================================================================

In [ ]:
# Define the weighted loss function

class_weights = torch.tensor([7.68, 2.15, 0.61, 0.47, 0.68, 6.26], dtype=torch.float32).to(device)
loss_fn       = BCEWithLogitsLoss(pos_weight=class_weights)  # For multi-label classification (binary classification per label)

## Class supports, class weigths, weighted loss function

#Reminder:
#*   df_jobs      : <class 'pandas.core.frame.DataFrame'>
#*   df_jobs['id']: <class 'pandas.core.series.Series'>

#dataset = Dataset.from_pandas(df_jobs)
#*   dataset      : <class 'datasets.arrow_dataset.Dataset'>
#*   dataset['id']: <class 'list'>

#*   dataset_dict_jobs : <class 'datasets.dataset_dict.DatasetDict'>
#*   train_dataset     : <class 'datasets.arrow_dataset.Dataset'>
#*   validation_dataset: <class 'datasets.arrow_dataset.Dataset'>
#*   test_dataset      : <class 'datasets.arrow_dataset.Dataset'>


#We calculate the class supports for the train, validation and test datasets; the class weights and the weighted loss function are used for training only; the class supports of validation_dataset and test_dataset are calculated for information only.
# function B
def get_train_class_weights(datasetDict, labels):
  print(f"datasetDict: {type(datasetDict)} shape={datasetDict.shape}\n{datasetDict}")
  print(f"labels: {type(labels)} len={len(labels)}\n{labels}")

  dataset_train      = datasetDict['train']
  dataset_validation = datasetDict['validation']
  dataset_test       = datasetDict['test']

  def calculate_class_supports(dataset, labels):
    class_supports = dataset.map(
        lambda example: {col: example[col] for col in labels},
        batched=True
    ).to_pandas()[labels].sum(axis=0)
    return class_supports

  class_supports = {}

  for split_name, split_dataset in datasetDict.items():
    class_supports[split_name] = calculate_class_supports(split_dataset, labels)

  for split_name, split_class_supports in class_supports.items():
    print(f"{split_name}: {type(split_class_supports)} len={len(split_class_supports)}\n{split_class_supports}")

  train_class_supports_list = class_supports['train'].tolist()
  print(f"train_class_supports_list: {type(train_class_supports_list)} len={len(train_class_supports_list)} {train_class_supports_list}")

  train_class_supports_tensor = torch.tensor(train_class_supports_list, dtype=torch.float32)
  print(f"train_class_supports_tensor: {type(train_class_supports_tensor)} len={len(train_class_supports_tensor)} {train_class_supports_tensor}")

  train_total_samples = dataset_train.num_rows
  print(f"train_total_samples: {train_total_samples}")

  number_of_classes = len(labels)
  print(f"number_of_classes: {number_of_classes}")

  train_class_weights = train_total_samples / (number_of_classes * train_class_supports_tensor)
  print(f"train_class_weights: {type(train_class_weights)} len={len(train_class_weights)} {train_class_weights}")

  train_class_weights_sum = train_class_weights.sum()
  print(f"train_class_weights_sum: {train_class_weights_sum}")

  normalized_train_class_weights = (train_class_weights / train_class_weights_sum) * number_of_classes
  print(f"normalized_train_class_weights: {type(normalized_train_class_weights)} len={len(normalized_train_class_weights)} {normalized_train_class_weights}")

  # Positives samples per label
  supports = train_class_supports_tensor
  print(f"supports: {type(supports)} {len(supports)} {supports}")

  # Negatives samples per label
  negatives = train_total_samples - supports
  print(f"negatives: {type(negatives)} {len(negatives)} {negatives}")

  # pos_weights = negative to positive ratios
  pos_weights = negatives/supports
  print(f"pos_weights: {type(pos_weights)} {len(pos_weights)} {pos_weights}")

  # Normalize using min-max scaling
  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  # Normalize using z-score standardization
  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  # Normalize using min-max scaling
  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  # Normalize using z-score standardization
  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  # Normalize using sum-to-one
  normalized_pos_weights_sum1 = pos_weights / pos_weights.sum()
  print(f"normalized_pos_weights_sum1: {type(normalized_pos_weights_sum1)} {len(normalized_pos_weights_sum1)} {normalized_pos_weights_sum1}")

  return normalized_pos_weights_minmax
  #return normalized_pos_weights_zscore
  #return normalized_pos_weights_sum1

pos_weights = get_train_class_weights(datasetDict, labels)

loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights.to(device))  # For multi-label classification (binary classification per label)
print(f"loss_fn: {type(loss_fn)} {loss_fn}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data from your training results
epochs = np.arange(1, 9)
training_loss = [0.3122, 0.2897, 0.2665, 0.2466, 0.2244, 0.2223, 0.2202, 0.2077]
validation_loss = [0.303785, 0.293599, 0.278830, 0.275663, 0.280968, 0.280640, 0.279608, 0.282026]
f1_score = [0.865113, 0.871222, 0.875554, 0.880279, 0.879128, 0.878554, 0.879872, 0.877893]

# Plot Loss
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, training_loss, label='Training Loss', marker='o')
plt.plot(epochs, validation_loss, label='Validation Loss', marker='s')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.grid()

# Plot F1 Score
plt.subplot(1, 2, 2)
plt.plot(epochs, f1_score, label='F1 Score', marker='o', color='green')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.title('F1 Score over Epochs')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
example = datasetDict['test'][0]
print(f"datasetDict['test'][0]: {type(example)} {example.keys()}\n{example}")

In [ ]:
inputs = tokenizer(
    example['text'],
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

In [ ]:
with torch.no_grad():  # Disable gradient calculation during prediction
    outputs = model(
        input_ids=inputs.input_ids.to(device),
        attention_mask=inputs.attention_mask.to(device)
    )


In [ ]:
probs = torch.sigmoid(outputs.logits)


In [ ]:
best_thresholds = [0.4, 0.3, 0.3, 0.3, 0.4, 0.5]
preds = np.zeros_like(probs)  # Initialize predictions array
for label_idx in range(num_labels):
  preds[:, label_idx] = (probs[:, label_idx] > best_thresholds[label_idx])  #.astype(int)

In [ ]:
print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")
print(f"preds: {type(preds)} shape={preds.shape}\n{preds}")

In [ ]:
print(f"labels: {type(labels)} {len(labels)}\n{labels}")
# '390': False, '135': False, '136': True, '137': True, '138': True, '139': False}

In [ ]:
def tune_thresholds(true_labels, probs, id2label):
  """
  Tune thresholds for each label to maximize F1 alone, as F1 balances precision and recall into a single metric.

  Args:
    true_labels: actual labels for the data                                      (numpy array of shape (num_samples, num_labels))
    probs      : predicted probabilities                                         (numpy array of shape (num_samples, num_labels))
    id2label   : dictionary mapping label indices (int) to label names (string)

  Returns:
    best_thresholds: best threshold for each label                                                      (numpy array of shape (num_labels,))
    best_metrics   : dictionary of best F1, precision_for_best_f1 and recall_for_best_f1 for each label (dictionary of numpy arrays)
  """
  thresholds      = np.linspace(0.1, 0.9, 9)
  best_thresholds = np.zeros(len(id2label))
  best_metrics    = {label: {'f1': 0.0, 'precision': 0.0, 'recall': 0.0} for label in id2label.values()}

  for label_idx, label in id2label.items():
    for threshold in thresholds:
      pred                     = (probs[:, label_idx] > threshold).astype(int)
      precision, recall, f1, _ = precision_recall_fscore_support(true_labels[:, label_idx], pred, average='binary', zero_division=0)
      if f1 > best_metrics[label]['f1']:
        best_thresholds[label_idx]       = threshold
        best_metrics[label]['f1']        = f1
        best_metrics[label]['precision'] = precision
        best_metrics[label]['recall']    = recall

  print("==== tune_thresholds ====")
  print(f"best_thresholds: {type(best_thresholds)} shape={best_thresholds.shape}\n{best_thresholds}")
  print(f"best_metrics   : {type(best_metrics)}    len={len(best_metrics)}      \n{json.dumps(best_metrics, indent=4)}")
  print("=========================")
  print()

  return best_thresholds, best_metrics

In [ ]:
def compute_metrics_with_threshold_OLD(probs, label_ids, thresholds, id2label):
    """
    Compute metrics during evaluation or test, by applying tuned thresholds

    average:
    - 'micro'   : gives more weight to frequent labels     → best for imbalanced datasets where frequent labels are more important
    - 'macro'   : treats all labels equally                → best when you care about rare labels as much as frequent ones
    - 'weighted': like macro but considers label frequency → best if you want a compromise between macro and micro

    - 'macro' or 'weighted' AUC is often best because AUC isn't as affected by class imbalance as F1/Precision/Recall
    - 'macro'      AUC: usually the best because it treats all labels equally, avoiding the dominance of frequent labels
    - 'weighted'   AUC: similar to macro but considers label frequency
    - 'macro'   PR AUC: best for imbalanced datasets because it treats rare labels fairly
    - 'weighted PR AUC: also good, but slightly biased toward frequent labels

    PR AUC is better than ROC AUC when you care about positive examples in imbalanced data
    """
    average = 'macro'
    preds   = np.zeros_like(probs)

    # Apply per-label tuned threshold
    for label_idx in id2label.keys():
        preds[:, label_idx] = (probs[:, label_idx] > thresholds[label_idx]).astype(int)

    # Compute metrics
    f1        = f1_score       (label_ids, preds, average=average)
    precision = precision_score(label_ids, preds, average=average)
    recall    = recall_score   (label_ids, preds, average=average)
    accuracy  = accuracy_score (label_ids, preds)

    # Compute AUC scores with error handling
    try:
        roc_auc              = roc_auc_score          (label_ids, probs, average=average)
    except ValueError:
        roc_auc              = 0.0

    try:
        precision_recall_auc = average_precision_score(label_ids, probs, average=average)
    except ValueError:
        precision_recall_auc = 0.0

    # Compute per-class metrics (average = None)
    per_class_f1        = f1_score       (label_ids, preds, average=None)
    per_class_precision = precision_score(label_ids, preds, average=None)
    per_class_recall    = recall_score   (label_ids, preds, average=None)

    # Generate classification report
    report = classification_report(label_ids, preds, target_names=id2label.values(), zero_division=0)

    # Store metrics
    metrics = {
        'f1'                   : f1,
        'precision'            : precision,
        'recall'               : recall,
        'accuracy'             : accuracy,
        'roc_auc'              : roc_auc,
        'precision_recall_auc' : precision_recall_auc,
        'thresholds'           : thresholds.tolist(),
        'classification_report': report,
        'per_class_f1'         : per_class_f1,
        'per_class_precision'  : per_class_precision,
        'per_class_recall'     : per_class_recall
    }

    return metrics

In [ ]:
def compute_metrics_with_threshold_SAV(probs, label_ids, thresholds, id2label):

  #Compute metrics during evaluation or test, by applying tuned thresholds

  #logits  = eval_preds.predictions
  #labels  = eval_preds.label_ids
  #sigmoid = torch.nn.Sigmoid  # Sigmoid or numpy?
  #probs   = sigmoid(logits).cpu().numpy()
  preds   = np.zeros_like(probs)

  if THRESHOLD_TUNING:
    # Apply per-label tuned threshold
    for label_idx in id2label.keys():
        preds[:, label_idx] = (probs[:, label_idx] > thresholds[label_idx]).astype(int)
  else:
    # threhsolds = None, apply a fixed threshold to all labels
    for label_idx in id2label.keys():
        preds[:, label_idx] = (probs[:, label_idx] > threshold).astype(int)

  # Compute metrics
  f1                    = f1_score               (label_ids, preds, average='micro')
  precision             = precision_score        (label_ids, preds, average='micro')
  recall                = recall_score           (label_ids, preds, average='micro')
  accuracy              = accuracy_score         (label_ids, preds)
  roc_auc               = roc_auc_score          (label_ids, probs, average='micro')  # multi_class="ovr" <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
  precision_recall_auc  = average_precision_score(label_ids, probs, average='micro')

  # Use id2label for target_names
  report = classification_report(label_ids, preds, target_names=id2label.values(), zero_division=0)

  if THRESHOLD_TUNING:
    _thresholds = thresholds.tolist()
  else:
    _thresholds = threshold

  metrics = {
      'f1'                   : f1,
      'precision'            : precision,
      'recall'               : recall,
      'accuracy'             : accuracy,
      'roc_auc'              : roc_auc,
      'precision_recall_auc' : precision_recall_auc,
      'thresholds'           : _thresholds,
      'classification_report': report
  }

  return metrics

In [ ]:
def predict_with_optimized_thresholds_cuda(trainer, dataset, threshold_tuning=False, thresholds=None, threshold=threshold):
    """
    Predicts using trainer.predict(), with optional threshold tuning on GPU

    Parameters:
    - trainer         : Hugging Face Trainer or CustomTrainer instance
    - dataset         : dataset to predict on
    - id2label        : dictionary mapping label indices (int) to label names (string)
    - threshold_tuning: boolean to enable thresholds tuning per class (if evaluation, True, if prediction, False)
    - thresholds      : if evaluation, custom thresholds, if prediction, tuned thresholds (from evaluation)

    Returns:
    - best_thresholds (if threshold_tuning=True): optimized threshold for each label
    - best_metrics                              : computed with tuned thresholds whether for evaluation or prediction

    - metrics (if threshold_tuning=False): computed with fixed thresholds
    - metrics (if threshold_tuning=True): computed with tuned thresholds
    - predictions: final binary predictions
    - label_ids  : ground true labels from the dataset
    - best_thresholds (if threshold_tuning=True): optimized threshold per class
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Auto-detect GPU

    # Predict
    predictions_output = trainer.predict(dataset)        # <class 'transformers.trainer_utils.PredictionOutput'> len   = 3
    predictions_np     = predictions_output.predictions  # <class 'numpy.ndarray'>                               shape = (2400, 6)
    label_ids_np       = predictions_output.label_ids    # <class 'numpy.ndarray'>                               shape = (2400, 6)
    metrics_dict       = predictions_output.metrics      # <class 'dict'> (= trainer.evaluate results)           len   = 10

    print("==== predictions_output ====")
    print(f"predictions_output.predictions: {type(predictions_np)} shape={predictions_np.shape} \n{predictions_np}")
    print(f"predictions_output.label_ids  : {type(label_ids_np)}   shape={label_ids_np.shape}   \n{label_ids_np}")
    print(f"predictions_output.metrics    : {type(metrics_dict)}   len={len(metrics_dict)}      \n{json.dumps(metrics_dict, indent=4)}")

    # Convert NumPy arrays to PyTorch tensors (torch.from_numpy() keeps the NumPy array's memory layout, while torch.tensor() creates a new copy)
    logits      = torch.tensor(predictions_np, device=device)  # Move to GPU
    true_labels = torch.tensor(label_ids_np, device=device)    # Move to GPU

    print(f"logits     : {type(logits)}      shape={logits.shape}     \n{logits}")
    print(f"true_labels: {type(true_labels)} shape={true_labels.shape}\n{true_labels}")

    # Convert logits to probabilities using PyTorch (on GPU)
    probs = torch.sigmoid(logits)  # <class 'torch.Tensor'>  shape = (1200, 6)

    print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")
    print("============================")
    print()

    num_labels = probs.shape[1]

    if THRESHOLD_TUNING:
        best_thresholds = torch.full((num_labels,), threshold, device=device, dtype=torch.float32)  # Default to threshold

        # Define candidate thresholds (on GPU)
        threshold_candidates = torch.linspace(0.05, 0.95, 19, device=device)

        for label_idx in range(num_labels):
            best_f1 = 0
            for threshold in threshold_candidates:
                preds = (probs[:, label_idx] > threshold).int()

                # Convert to CPU for sklearn
                precision, recall, f1, _ = precision_recall_fscore_support(
                    true_labels[:, label_idx].cpu().numpy(),
                    preds.cpu().numpy(),
                    average='binary',
                    zero_division=0
                )

                if f1 > best_f1:
                    best_f1                    = f1
                    best_thresholds[label_idx] = threshold  # Store best threshold

        return probs, true_labels, best_thresholds

    # Apply provided thresholds (or default to 0.5)
    if thresholds is None:
        thresholds = torch.full((num_labels,), 0.5, device=device)  # Default to 0.5
    else:
        thresholds = torch.tensor(thresholds, device=device)  # Move thresholds to GPU

    preds = (probs > thresholds).int()

    return preds, true_labels, None

In [ ]:
#test_trainer_predict_results = predict_with_optimized_thresholds(
#    trainer, test_dataset, id2label, threshold_tuning=False, thresholds=optimized_thresholds)

#except_report = {k: v for k, v in test_trainer_predict_results.items() if k!='classification_report'}
#report        = test_trainer_predict_results['classification_report']
#print(f"test_trainer_predict_results: {type(except_report)} len={len(except_report)}\n{json.dumps(except_report, indent=4)}")
#print(f"test_trainer_predict_results['classification_report']: {type(report)} len={len(report)}\n{report}")
